In [1]:
# Required Lib

import random
import math
from math import log, exp, e
from datetime import datetime, timedelta
import unicodedata       

In [2]:
# USER & PRODUCT Backend Logic
# Product

def product_age_adjust(days_since_creation: int):
  age_adjustment = 3 -(days_since_creation) / 25
  return max(age_adjustment,-10)

def product_inventory_adjust(avg_inventory: int, min_limit=-5):
    if avg_inventory < 0:
        avg_inventory = 0
    adjustment = 15 - (5 * log(avg_inventory + 1))
    return max(adjustment, min_limit)

# def product_inventory_adjust(avg_inventory: int):
#   inventory_adjustment = 15 - (5 * log(avg_inventory + 1))
#   return max(inventory_adjustment,-5)

def produt_image_adjust(image_count: int):
  image_bonus = 2 * min(image_count, 5)
  return image_bonus

def product_type_adjust(product_type):
  if product_type == 'seasonal':
    product_type_adjust = 5
  else:
    product_type_adjust = 0
  return product_type_adjust

seasonal_tags = ["winter", "summer", "spring", "autumn"]
def product_tag_adjust(product_tag):
  if product_tag in seasonal_tags:
    product_tag_adjust = 5
  else:
    product_tag_adjust = 0
  return product_tag_adjust

def avg_price_adjust(avg_price: float):
  price_adjustment = min(15,3 * log(avg_price + 1) - 10)
  return price_adjustment


def calculate_product_score(days_since_creation , avg_inventory, image_count, product_type, product_tag, avg_price):
  age_adjustment = product_age_adjust(days_since_creation )
  inventory_adjustment = product_inventory_adjust(avg_inventory)
  image_bonus = produt_image_adjust(image_count)
  product_type = product_type_adjust(product_type)
  product_tag = product_tag_adjust(product_tag)
  price_adjustment = avg_price_adjust(avg_price)
  return max(age_adjustment + inventory_adjustment + image_bonus + product_type + product_tag + price_adjustment,0) 


# User score 

def user_order_score(order_count):
  order_score =min(15, 3 *log(order_count + 1))
  return order_score

# replace with user_cancelled_order_penalty
def user_refund_penalty(total_refunded, total_spent):
  refund_penalty = -10 * (total_refunded / (total_spent + 1))
  return refund_penalty

def user_recency_bonus(days_since_last_purchase):
  recency_bonus = 10 * exp(-days_since_last_purchase / 100)
  return recency_bonus

def user_spending_adjust(total_spent):
  return 15 * (2 / (1 + exp(-0.002 * (total_spent - 500))) - 1)

def user_email_verify_adjust(verified_email):
  if verified_email:
        return 5
  else:
    return 0


def calculate_user_score(order_count, total_refunded, total_spent, days_since_last_purchase, verified_email):
  order_score = user_order_score(order_count)
  refund_penalty = user_refund_penalty(total_refunded, total_spent)
  recency_bonus = user_recency_bonus(days_since_last_purchase)
  spending = user_spending_adjust(total_spent)
  verify_email = user_email_verify_adjust(verified_email)
  return order_score + refund_penalty + recency_bonus + spending + verify_email

In [3]:
# Max round calculation Logic

MAX_USER_SCORE = 45
MAX_PRODUCT_SCORE = 53
MAX_BARGAIN_SCORE = 100

def calculate_max_rounds(user_score, product_score, bargain_score):
  # Normalize user and bargain scores between 0 and 1
  user_factor = user_score / MAX_USER_SCORE
  bargain_factor = bargain_score / MAX_BARGAIN_SCORE
  # Use an inverse relation for product influence (higher product score reduces rounds)
  product_factor = 1 / (product_score + 1)  # Avoid division by zero
  max_rounds = min(6, round((user_factor + product_factor + bargain_factor) * 3) + 2)

  return max_rounds

In [4]:
# Max allowed discount based on different scores 

def adjust_max_discount (admin_max_discount, user_score, product_score,bargain_score, max_user_score=45, max_product_score=53):
  # Normalize user influence (range: 0 to 1)
  user_factor = user_score / max_user_score
  # Scale product influence (higher product score reduces discount, max 30% reduction)
  product_factor = 1 - (product_score / max_product_score) * 0.3
  # Adjust bargain factor with a 30% effect
  normalized_bargain_score = max(0.5, min(1, bargain_score / 65))
  bargain_factor = 1 - 0.3 * (1 - normalized_bargain_score)  # Only 30% effect
  adjusted_discount = admin_max_discount * user_factor * product_factor * bargain_factor
  adjusted_discount = math.ceil(adjusted_discount)
    
  return max(adjusted_discount, 0.5*admin_max_discount) #ensures that discount never goes zero

In [5]:
# Get discounts for all rounds

def calculate_round_discount(max_round, round_count, max_disc):
    if round_count < 0 or round_count > max_round:
        raise ValueError("round_count must be between 0 and max_round")

    # Ensure a smooth logarithmic increase, avoiding zero at round_count=0
    base_discount = max_disc * 0.1  # 10% of max discount as a base
    scaling_factor = log(round_count + 1) / log(max_round + 1)

    # Calculate discount dynamically, ensuring it starts small but > 0
    discount = base_discount + (scaling_factor * (max_disc - base_discount))

    return math.floor(discount)  # Round down to the nearest integer

def generate_all_round_discounts(max_round, max_disc):
    round_discounts = {}

    for round_count in range(1, max_round + 1):
        discount = calculate_round_discount(max_round, round_count, max_disc)
        round_discounts[f'round_count{round_count}'] = discount  # Key format

    return round_discounts

In [6]:
# Handle User Email and Product details

def get_product_detail(product_name, product_descriptions, product_sizes, color):
    # Get the selected product description
    description = product_descriptions.get(product_name) #, "Description not available")
    # Get the available sizes for the selected product
    sizes = product_sizes.get(product_name) #, ["One Size"])
    # Select a random size from the available sizes
    selected_size = random.choice(sizes)
    # Get the selected random color
    product_color = color
    
    # Output the selected product and size
    return {
        "Product_Name": product_name,
        "Product_Description": description,
        "Product_Size": selected_size,
        "Product_color": product_color
}


def user_email_handler(user_name):
    if user_name:
        user_email = f"{user_name.lower()}@gmail.com"
    else:
        user_email = user_name

    return user_email

In [7]:
# Admin Minimum Price Handler

def get_min_price_from_product_price(product_price):
    max_discount_percentage = random.randint(10, 60)
    #print(f"max_discount_percentage : {max_discount_percentage}")
    discount_amount = product_price * (max_discount_percentage / 100)
    #print(f"discount_amount:{discount_amount}")

    min_price = round(product_price - discount_amount)
    #print(f"min_price:{min_price}")
    return min_price, max_discount_percentage

In [8]:
# Current round discounted number
def get_discount_for_current_round(current_round_count, discount_round_count):
    # Construct the key name dynamically
    key = f'round_count{current_round_count}'
    # Get the corresponding discount
    current_discount = discount_round_count.get(key)
    return current_discount


# Get price for the discounted number
def get_original_price_from_discounts(Original_Price, Discount_Percentage):
    Discounted_original_Price = Original_Price * (1 - Discount_Percentage / 100)
    return Discounted_original_Price 

# Get previous round discount and user re for scenario gen
def get_previous_round(current_round, round_counts_with_discounts):
    previous_round_key = f'round_count{current_round - 1}'
    previous_round_discount = round_counts_with_discounts.get(previous_round_key, None)
    user_add = random.randint(9, 29)
    #print(user_add)
    user_req = previous_round_discount + user_add
    return previous_round_discount, user_req

In [9]:
# To get round count between 3 to 6 

def get_lower_bound(max_round_count):
    if max_round_count <= 3:
        lower_bound = 3
    elif max_round_count >= 4:
        lower_bound = 4
    return lower_bound

In [10]:
# for generating clean dataset

def normalize_text(text):
    if isinstance(text, str):
        return unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8")
    return text  # Return as-is if not a string


# # Normalize key text fields
# user = normalize_text(user)
# response = normalize_text(response)
# Data_from_data_base = normalize_text(Data_from_data_base)

# Use standard json.dumps — no need for ensure_ascii=False 
# or
# print(json.dumps(dataset, indent=2, ensure_ascii=False))

In [14]:
# Main Function

def generate_scenario():
    
    user_name = random.choice([
    "Ramesh", "Sneha", "Aman", "Priya", "Nikhil", "Ayesha", "Zara", "Kiran", "Manav", "Diya",
    "Arjun", "Ananya", "Ishaan", "Kiara", "Neha", "Aarav", "Meera", "Vivaan", "Saanvi", "Devika",
    "Rahul", "Anika", "Dhruv", "Kavya", "Riya", "Aarohi", "Tanvi", "Yashvi", "Eshaan", "Shanaya",
    "Manish", "Avani", "Bhavya", "Chetan", "Deepak", "Anjali", "Girish", "Pooja", "Vidhi", "", None])
    
    user_email = user_email_handler(user_name)
    
    product_name = random.choice([
    "Shirt", "Shoes", "Watch", "Jacket", "Backpack", "Sunglasses", "Sweater", "Kurta", "Jeans", "T-shirt",
    "Scarf", "Hat", "Sneakers", "Boots", "Blazer", "Wallet", "Gloves", "Belt", "Tie", "Cufflinks",
    "Chinos", "Cardigan", "Pajamas", "Shorts", "Raincoat", "Sweatshirt", "Tracksuit", "Vest", "Overcoat",
    "Sandals", "Flip-flops", "Beanie", "Headband", "Socks", "Umbrella", "Luggage", "Duffel Bag", "Laptop Sleeve",
    # Electronic
    "Smartphone", "Laptop", "Tablet", "Smartwatch", "Bluetooth Headphones", "Wireless Earbuds", "Smart TV", "Digital Camera", "Portable Speaker", "Power Bank",
    "Electric Kettle", "Hair Dryer", "Electric Toothbrush", "Electric Shaver", "Microwave Oven", "Refrigerator", "Washing Machine", "Air Conditioner", "Water Purifier", "Electric Fan",
    "Induction Cooktop", "Electric Pressure Cooker", "Electric Grill", "Electric Rice Cooker", "Electric Juicer", "Electric Mixer Grinder", "Electric Iron", "Electric Heater", "Electric Blanket", "Electric Stove",
    "Electric Oven", "Electric Chimney", "Electric Geyser", "Electric Water Heater", "Electric Cooker", "Electric Deep Fryer", "Electric Sandwich Maker", "Electric Popcorn Maker", "Electric Coffee Maker", "Electric Tea Maker", "Electric Milk Frother",
    "Projector", "Smart Home Hub", "Wi-Fi Router", "Smart Doorbell", "Smart Lock", "Security Camera",
    # Home & Living / Furniture & Décor
    "Wall Art", "Cushion", "Rug", "Table Lamp", "Indoor Plant", "Coffee Table", "Dining Table", "Bookshelf", "Office Chair",
    "Sofa", "Bed Frame", "Mattress", "Nightstand", "Wardrobe", "TV Stand", "Curtains", "Blinds", "Mirror", "Vase",
    "Candles", "Throw Blanket", "Bookshelf Speaker", "Area Heater", "Air Purifier", "Humidifier", "Smart Thermostat", "Robot Vacuum Cleaner", "Steam Mop", "Storage Cabinet",
    # Kitchenware & Dining
    "Cutlery Set", "Frying Pan", "Dinnerware Set", "Chef's Knife", "Cutting Board", "Mixer", "Stand Mixer", "Blender", "Food Processor", "Slow Cooker",
    "Pressure Cooker", "Coffee Grinder", "Espresso Machine", "Toaster", "Toaster Oven", "Teapot", "Bakeware", "Glassware", "Barware", "Spice Rack",
    "Knife Block", "Dish Rack", "Mortar & Pestle", "Rolling Pin", "Measuring Cups", "Measuring Spoons", "Dish Towels", "Oven Mitts", "Ice Maker", "Wine Cooler",
    "Water Dispenser", "Portable BBQ Grill",
    # Beauty, Personal Care & Health
    "Face Cream", "Lipstick", "Shampoo Bottle", "Hair Serum", "Body Lotion", "Body Butter", "Sunscreen", "Perfume", "Makeup Palette", "Foundation",
    "Mascara", "Eyeliner", "Nail Polish", "Electric Toothbrush", "Flosser", "Electric Shaver", "Trimmer", "Hair Dryer", "Straightener", "Curling Iron",
    "Electric Razor", "Epilator", "Eyelash Curler", "Hair Dryer Diffuser", "Facial Cleansing Brush", "Electric Massager", "Foot Spa",
    "First Aid Kit", "Vitamins", "Supplements", "Fitness Tracker", "Smart Scale", "Yoga Mat", "Resistance Bands", "Treadmill", "Exercise Bike",
    "Massage Gun", "Yoga Blocks", "Protein Powder", "Nutrition Bars", "Blood Pressure Monitor", "Glucometer", "Thermometer", "Pulse Oximeter",
    # Food & Beverage, Grocery & Gourmet
    "Organic Olive Oil", "Gourmet Chocolate", "Artisanal Cheese", "Herbal Tea", "Coffee Beans", "Ground Coffee", "Spices", "Seasonings", "Organic Honey", "Maple Syrup",
    "Snack Packs", "Nutrition Bars", "Frozen Foods", "Vegan Products", "Plant-Based Meat", "Health Foods", "Supplements", "Craft Beer", "Wine", "Spirits",
    "Cold Pressed Juice", "Smoothie Packs", "Gourmet Snacks", "Bakery Items", "Condiments", "Sauces", "Meal Kits", "Pet Food",
    # DIY & Hardware / Furniture & Lumber
    "Lawn Mower", "Garden Hose", "Outdoor Furniture Set", "BBQ Grill", "Electric Drill", "Impact Driver", "Hammer", "Crowbar", "Screwdriver Set", "Bit Kit",
    "Measuring Tape", "Laser Distance Meter", "Socket Set", "Wrench Set", "Tool Chest", "Storage Cabinet", "Paint", "Brushes", "Power Saw", "Jigsaw",
    "Nails", "Screws", "Lumber", "Plywood", "Safety Gear", "Work Light", "Flashlight", "Extension Cord", "Power Strip", "Tool Belt",
    "Apron", "Paint Rollers", "Ladders", "Step Stool",
    # Office & School Supplies
    "Notebook", "Journal", "Pen Set", "Fountain Pen", "Desk Organizer", "File Holder", "Stapler", "Staple Remover", "Printer", "Scanner",
    "Mouse Pad", "Wrist Rest", "Desk Lamp", "Task Light", "Whiteboard", "Corkboard", "Calculator", "Financial Calculator", "File Cabinet", "Bookends",
    "Binder", "Dividers", "Backpack", "Laptop Bag", "Markers", "Highlighters", "Paper", "Envelopes", "Sticky Notes", "Index Tabs",
    "Label Maker", "Labels", "USB Flash Drive", "External Hard Drive", "Power Bank", "Charging Station", "Webcam", "Headset",
    # Sports, Fitness & Outdoor
    "Dumbbell Set", "Kettlebell", "Cycling Helmet", "Protective Gear", "Camping Tent", "Sleeping Bag", "Hiking Backpack", "Trekking Poles", "Fishing Rod", "Reel",
    "Golf Clubs", "Golf Bag", "Tennis Racket", "Tennis Balls", "Basketball", "Soccer Ball", "Camping Stove", "Cookware", "Inflatable Kayak", "Paddleboard",
    "Outdoor Furniture", "Hammock", "Pool Accessories", "Beach Accessories", "Climbing Gear", "Backpacking Gear", "Golf Apparel", "Golf Shoes",
    # Toys, Games & Collectibles
    "Action Figure", "Collectible Figurine", "Board Game", "Card Game", "Puzzle", "Jigsaw Puzzle", "Dollhouse", "Miniature Sets", "Model Kit", "RC Vehicle",
    "Lego Set", "Building Blocks", "Stuffed Animal", "Plush Toy", "Educational Toy", "STEM Kit", "Video Game Console", "Game Controller", "Arcade Stick", "VR Headset",
    "Gaming Chair", "Gaming Desk Mat", "Collectible Card", "Trading Card", "Toy Storage", "Organizer", "Outdoor Play Equipment", "Swing Set", "Drone", "RC Helicopter",
    # Automotive & Industrial
    "Car Cover", "Sunshade", "Steering Wheel Cover", "Seat Covers", "GPS Navigator", "Dash Cam", "Car Vacuum", "Portable Air Compressor", "Tire Inflator", "Jump Starter",
    "Floor Mats", "Trunk Organizer", "Car Audio", "Speakers", "Motor Oil", "Fluids", "Brake Pads", "Rotors", "Car Battery", "Battery Charger",
    "Roof Rack", "Cargo Carrier", "Tow Hitch", "Trailer Hitch", "Safety Hammer", "Seat Belt Cutter", "Headlight Assembly", "Taillight Assembly", "Spark Plug", "Ignition Coil",
    "Air Filter", "Cabin Filter", "Jumper Cables", "Automotive Tool Kit", "Industrial Lubricants", "Grease", "3D Printer", "Filament", "CNC Accessories", "Router Bits",
    # Pet Supplies & Services
    "Dog Leash", "Collar", "Cat Toy", "Laser Pointer", "Pet Bed", "Crate", "Pet Food Bowl", "Feeder", "Gourmet Pet Treats", "Grooming Kit",
    "Aquarium", "Fish Tank", "Bird Cage", "Perch", "Small Animal Hutch", "Cage", "Pet Carrier", "Travel Crate", "Flea Preventive", "Tick Preventive",
    "Dewormer", "Pet Clothing", "Accessories", "Pet ID Tag", "Microchip Service", "Pet Insurance", "Veterinary Plan", "Training Classes", "Behavior Consultation",
    # Books, Media & Digital Content
    "Paperback", "Hardcover", "Audiobook", "E-Book", "Digital Download", "Magazine Subscription", "Journal Subscription", "Music Album", "Vinyl Record", "Blu-ray",
    "DVD", "4K UHD", "Streaming Service Subscription", "Video Game Title", "Digital DLC", "Software License", "Productivity Suite", "Cloud Storage Subscription", "Online Course", "e-Learning Platform",
    "Virtual Conference Ticket", "Webinar Pass", "Stock Photos", "Graphics Pack", "NFT", "Digital Collectible",
    # Art Supplies & Crafting
    "Canvas Set", "Easel", "Paint Brushes", "Palette Knives", "Acrylic Paint", "Oil Paint", "Watercolor Paint", "Sketchbook", "Drawing Pad", "Watercolor Palette",
    "Colored Pencils", "Markers", "Crafting Kit", "DIY Project Box", "Sewing Machine", "Accessories", "Scrapbooking Supplies", "Stickers", "Beads", "Jewelry Making Supplies",
    "Pottery Tools", "Clay", "3D Printer", "Filament", "Digital Drawing Tablet", "Stylus", "Studio Lighting",
    # Baby & Toddler
    "Baby Stroller", "Travel System", "Diapers", "Pull-Ups", "Baby Monitor", "Video Monitor", "Baby Lotion", "Baby Shampoo", "Crib", "Bassinet",
    "Changing Table", "Changing Pad", "Baby Carrier", "Wrap", "Sling", "High Chair", "Booster Seat", "Playard", "Pack & Play", "Toddler Toy",
    "Educational Toy", "Baby Clothes", "Bodysuits", "Pacifier", "Teethers", "Car Seat", "Booster Seat", "Baby Bottle", "Sterilizer", "Breast Pump",
    "Nursing Pillow", "Diaper Bag", "Backpack", "Changing Mat", "Baby Blanket", "Swaddle", "Baby Shoes", "Booties", "Safety Gate", "Corner Guards",
    # Jewelry & Watches
    "Necklace", "Pendant", "Earrings", "Studs", "Hoops", "Bracelet", "Bangles", "Ring", "Band", "Anklet",
    "Toe Ring", "Brooch", "Lapel Pin", "Cufflinks", "Tie Clip", "Jewelry Organizer", "Box", "Watch Winder", "Watch Case",
    # Party & Festivity
    "Party Lights", "String Lights", "Inflatable Decorations", "Yard Inflatables", "Birthday Banner", "Custom Banner", "Confetti Pack", "Tableware", "Disposable Plates", "Helium Tank",
    "Balloons", "Photo Booth Props", "Backdrops", "Party Favors", "Goodie Bags", "Cake Topper", "Dessert Stand", "Sound System", "DJ Equipment", "Event Tickets",
    "Passes",
    # Video Games & Virtual Reality
    "Gaming Console", "Hardware", "Game Controller", "Arcade Stick", "Racing Wheel", "Pedals", "VR Headset", "Accessories", "Trackers", "Haptic Gloves",
    "Gaming Chair", "Racing Simulator", "Gaming Headset", "Microphone", "Monitor", "Gaming Monitor", "Graphics Card", "GPU", "Gaming PC", "Components",
    "Gaming Laptop", "Portable Gaming", "Streaming Gear", "Capture Card", "Green Screen", "Ring Light", "Game Title", "Digital DLC", "Game Subscription", "Online Service",
    "In-Game Currency",
    # Software, Subscriptions & Digital Services
    "Antivirus Software", "Security Software", "Photo Editing Software", "Video Editing Software", "Office Suite", "Productivity Tools", "Cloud Storage", "Backup Service", "Website Hosting", "Domain Service",
    "VPN Subscription", "Streaming Service", "Digital Marketing Tool", "SEO Software", "Project Management Software", "E-Book Subscription", "Audiobook Service", "VPS", "Cloud Compute", "SaaS Tool",
    # Eco-Friendly & Sustainable Products
    "Reusable Water Bottle", "Tumblers", "Biodegradable Straws", "Cutlery", "Reusable Grocery Bags", "Produce Bags", "Solar Charger", "Solar Panel", "Bamboo Toothbrush", "Dental Floss",
    "Beeswax Wraps", "Reusable Food Wraps", "Eco-Friendly Cleaning Products", "Compost Bin", "Compost Tumbler", "Organic Cotton Towels", "Bedsheets", "Sustainable Fashion", "Ethical Apparel", "Plantable Stationery",
    # Luxury Goods & Premium Segments
    "Luxury Handbag", "Designer Purse", "High-End Watch", "Swiss Timepiece", "Premium Spirit", "Aged Whiskey", "Designer Clothing", "Haute Couture", "Fine Jewelry", "Gemstone Ring",
    "Collector Car", "Luxury Automobile", "Art & Antique", "Limited Edition Print", "Yacht", "Private Jet", "Custom-Built Home",
    # Recommerce & Pre-Owned / Refurbished
    "Refurbished Electronic", "Certified Pre-Owned Device", "Pre-Owned Luxury Bag", "Designer Apparel", "Used Book", "Collectible Edition", "Vintage Vinyl Record", "Secondhand Instrument"])

    product_descriptions = {
    # Apparel & Accessories
    "Shirt": "A classic button-down shirt made from breathable cotton, perfect for both casual and formal occasions.",
    "Shoes": "Stylish leather shoes that offer comfort and durability, ideal for daily wear.",
    "Watch": "A sleek stainless steel watch with a minimalist design, featuring water resistance and a date function.",
    "Jacket": "A lightweight bomber jacket with ribbed cuffs and hem, suitable for layering in cooler weather.",
    "Backpack": "Spacious and ergonomic backpack with multiple compartments, designed for daily commuting.",
    "Sunglasses": "UV-protective sunglasses with polarized lenses, combining fashion and functionality.",
    "Sweater": "Soft woolen sweater with a cozy fit, perfect for chilly evenings.",
    "Kurta": "Traditional Indian kurta made from cotton, featuring intricate embroidery and a relaxed fit.",
    "Jeans": "Denim jeans with a slim fit and stretchable fabric, offering both style and comfort.",
    "T-shirt": "Cotton T-shirt with a graphic print, available in various colors.",
    "Scarf": "Lightweight scarf made from silk, adding a touch of elegance to any outfit.",
    "Hat": "Wide-brimmed hat providing sun protection and a stylish look.",
    "Sneakers": "Comfortable sneakers with cushioned soles, suitable for running and casual outings.",
    "Boots": "Leather boots with a rugged design, perfect for outdoor adventures.",
    "Blazer": "Tailored blazer with a modern cut, ideal for professional settings.",
    "Wallet": "Genuine leather wallet with multiple card slots and a coin pocket.",
    "Gloves": "Thermal gloves designed to keep your hands warm during winter.",
    "Belt": "Classic leather belt with a polished buckle, complementing formal attire.",
    "Tie": "Silk tie with a subtle pattern, adding sophistication to your outfit.",
    "Cufflinks": "Elegant cufflinks crafted from sterling silver, enhancing your shirt cuffs.",
    "Chinos": "Cotton chinos with a slim fit, versatile for both work and casual wear.",
    "Cardigan": "Button-up cardigan made from soft cotton, perfect for layering.",
    "Pajamas": "Comfortable cotton pajamas set, ensuring a good night's sleep.",
    "Shorts": "Casual shorts with an adjustable waistband, suitable for warm weather.",
    "Raincoat": "Waterproof raincoat with a hood, keeping you dry during showers.",
    "Sweatshirt": "Cotton sweatshirt with a fleece lining, offering warmth and comfort.",
    "Tracksuit": "Matching tracksuit with zippered pockets, ideal for workouts.",
    "Vest": "Lightweight vest with a zip closure, suitable for layering.",
    "Overcoat": "Woolen overcoat with a double-breasted design, perfect for winter.",
    "Sandals": "Comfortable sandals with adjustable straps, suitable for summer outings.",
    "Flip-flops": "Rubber flip-flops with a cushioned footbed, ideal for beach trips.",
    "Beanie": "Knit beanie hat, keeping your head warm during cold weather.",
    "Headband": "Stretchable headband, perfect for workouts or casual wear.",
    "Socks": "Cotton socks with reinforced heels and toes, ensuring durability.",
    "Umbrella": "Compact umbrella with a wind-resistant frame, easy to carry.",
    "Luggage": "Hard-shell luggage with spinner wheels, offering smooth mobility.",
    "Duffel Bag": "Spacious duffel bag with a detachable shoulder strap, suitable for travel.",
    "Laptop Sleeve": "Padded laptop sleeve with a zipper closure, protecting your device.",
     # Electronic   
    "Smartphone": "A sleek smartphone with a high-resolution camera, long-lasting battery, and a fast processor, perfect for everyday use.",
    "Laptop": "A lightweight laptop featuring a powerful processor, ample storage, and a high-definition display, ideal for work and entertainment.",
    "Tablet": "A versatile tablet with a large screen, responsive touch interface, and long battery life, suitable for reading, browsing, and streaming.",
    "Smartwatch": "A stylish smartwatch that tracks your fitness, monitors your health, and keeps you connected with notifications and calls.",
    "Bluetooth Headphones": "Wireless headphones offering immersive sound quality, noise cancellation, and a comfortable fit for all-day wear.",
    "Wireless Earbuds": "Compact earbuds with touch controls, sweat resistance, and superior sound quality for an on-the-go lifestyle.",
    "Smart TV": "A smart television with built-in streaming apps, voice control, and high-definition picture quality for an enhanced viewing experience.",
    "Digital Camera": "A high-resolution digital camera with optical zoom, image stabilization, and various shooting modes for capturing memories.",
    "Portable Speaker": "A waterproof portable speaker with Bluetooth connectivity, delivering rich sound and deep bass for outdoor adventures.",
    "Power Bank": "A compact power bank with fast charging capabilities, ensuring your devices stay powered throughout the day.",
    "Electric Kettle": "A stainless steel electric kettle with rapid boiling technology, auto shut-off feature, and a sleek design.",
    "Hair Dryer": "A lightweight hair dryer with multiple heat settings, ionic technology, and a cool shot button for a smooth finish.",
    "Electric Toothbrush": "An electric toothbrush with sonic technology, multiple brushing modes, and a timer for effective oral care.",
    "Electric Shaver": "A cordless electric shaver with hypoallergenic blades, providing a close and comfortable shave.",
    "Microwave Oven": "A microwave oven with multiple cooking presets, defrost function, and a spacious interior for quick meal preparation.",
    "Refrigerator": "A double-door refrigerator with energy-efficient cooling, adjustable shelves, and a spacious freezer compartment.",
    "Washing Machine": "A front-load washing machine with multiple wash programs, energy efficiency, and a large capacity drum.",
    "Air Conditioner": "A split air conditioner with inverter technology, providing efficient cooling and low noise operation.",
    "Water Purifier": "A multi-stage water purifier with UV and RO filtration, ensuring safe and clean drinking water.",
    "Electric Fan": "A high-speed electric fan with oscillation, multiple speed settings, and a sleek design for optimal air circulation.",
    "Induction Cooktop": "A portable induction cooktop with touch controls, multiple power levels, and a sleek glass surface.",
    "Electric Pressure Cooker": "A programmable electric pressure cooker with multiple cooking presets, ensuring quick and easy meal preparation.",
    "Electric Grill": "An electric grill with non-stick plates, adjustable temperature control, and a drip tray for healthier cooking.",
    "Electric Rice Cooker": "A rice cooker with one-touch cooking, keep-warm function, and a non-stick inner pot for easy cleaning.",
    "Electric Juicer": "A centrifugal juicer with wide feeding tube, stainless steel blades, and anti-drip spout for fresh juice.",
    "Electric Mixer Grinder": "A powerful mixer grinder with multiple jars, stainless steel blades, and overload protection for versatile blending.",
    "Electric Iron": "A steam iron with ceramic soleplate, vertical steam function, and anti-drip system for wrinkle-free clothes.",
    "Electric Heater": "A compact electric heater with adjustable thermostat, overheat protection, and quiet operation for cozy warmth.",
    "Electric Blanket": "A soft electric blanket with multiple heat settings, auto shut-off feature, and machine-washable fabric.",
    "Electric Stove": "A portable electric stove with multiple burners, adjustable heat controls, and a compact design for easy storage.",
    "Electric Oven": "A countertop electric oven with convection cooking, multiple rack positions, and a timer for precise baking.",
    "Electric Chimney": "A wall-mounted electric chimney with powerful suction, LED lighting, and easy-to-clean filters for a smoke-free kitchen.",
    "Electric Geyser": "A high-efficiency electric geyser with fast heating, temperature control, and safety features for hot water.",
    "Electric Water Heater": "A tankless electric water heater with instant heating, energy-saving mode, and compact design for space-saving.",
    "Electric Cooker": "A multi-functional electric cooker with rice cooking, steaming, and slow cooking capabilities in one appliance.",
    "Electric Deep Fryer": "A deep fryer with adjustable temperature control, large capacity, and oil filtration system for crispy fried foods.",
    "Electric Sandwich Maker": "A sandwich maker with non-stick plates, adjustable browning control, and cool-touch handles for safe use.",
    "Electric Popcorn Maker": "A hot air popcorn maker with no oil required, producing healthy and fluffy popcorn in minutes.",
    "Electric Coffee Maker": "A programmable coffee maker with brew strength control, auto-shut off, and a reusable filter for fresh coffee.",
    "Electric Tea Maker": "An electric tea maker with temperature control, keep-warm function, and stainless steel infuser for perfect tea.",
    "Electric Milk Frother": "A milk frother with multiple frothing options, stainless steel whisk, and easy-clean design for creamy beverages.",
    "Projector": "Portable projector with 1080p support, ideal for home theaters or business presentations.",
    "Smart Home Hub": "Central hub to connect and control various smart devices (lights, locks, thermostats) via voice or app.",
    "Wi-Fi Router": "Dual-band router with MU-MIMO technology, providing fast and reliable wireless coverage.",
    "Smart Doorbell": "Video doorbell with 1080p camera, motion detection, and two-way audio for home security.",
    "Smart Lock": "Keyless door lock with fingerprint, PIN, or smartphone app access for secure home entry.",
    "Security Camera": "Indoor/outdoor IP camera with night vision, motion alerts, and cloud storage options.",
    # Home & Living / Furniture & Décor
    "Wall Art": "Canvas or framed artwork to add personality and color to any room.",
    "Cushion": "Plush cushion with removable cover, providing comfort and style for sofas or beds.",
    "Rug": "Textured area rug made from durable materials, creating a cozy ambiance in living spaces.",
    "Table Lamp": "Adjustable LED table lamp with touch controls and multiple brightness levels.",
    "Indoor Plant": "Low-maintenance potted plant (e.g., succulent, snake plant) to purify air and enhance decor.",
    "Coffee Table": "Modern or vintage coffee table in wood, glass, or metal to anchor your living room.",
    "Dining Table": "Wooden, glass, or marble dining table that seats four to eight people.",
    "Bookshelf": "Freestanding or wall-mounted shelves in multiple tiers for books and decor.",
    "Office Chair": "Ergonomic chair with adjustable lumbar support, armrests, and height settings for comfort.",
    "Sofa": "Sectional, loveseat, or sleeper sofa upholstered in fabric or leather for cozy lounging.",
    "Bed Frame": "Platform or storage bed frame in twin, queen, king, or California king sizes.",
    "Mattress": "Memory foam, innerspring, hybrid, or latex mattress in various firmness levels.",
    "Nightstand": "Bedside table with drawers or open shelves in wood or metal finishes.",
    "Wardrobe": "Freestanding wardrobe or modular closet organizer with hanging rods and shelves.",
    "TV Stand": "Media console with cable management to neatly house your television and devices.",
    "Curtains": "Blackout or sheer curtains in lengths like 84″, 96″, or 108″ to control light and privacy.",
    "Blinds": "Roller, cellular, or Venetian blinds with motorized or manual operation.",
    "Mirror": "Wall-mounted or floor-standing mirror with decorative frame to enhance room size perception.",
    "Vase": "Ceramic, glass, or metal vase for fresh or faux flower arrangements.",
    "Candles": "Scented or unscented candles in jars, pillars, or tapers with matching holders.",
    "Throw Blanket": "Knit or fleece throw for extra warmth and a pop of texture on sofas or beds.",
    "Bookshelf Speaker": "Compact Hi-Fi speaker for high-quality audio in living or workspace.",
    "Area Heater": "Wall-mounted or pedestal heater as a supplemental heating source in any room.",
    "Air Purifier": "HEPA filter air purifier to remove allergens, dust, and pollutants from indoor air.",
    "Humidifier": "Ultrasonic or evaporative humidifier to maintain balanced humidity levels.",
    "Smart Thermostat": "Wi-Fi thermostat that learns your schedule for energy-efficient home temperature control.",
    "Robot Vacuum Cleaner": "Autonomous vacuum with mapping, scheduling, and app control for hands-free cleaning.",
    "Steam Mop": "Steam mop with microfiber pads for deep cleaning of hard floors.",
    "Storage Cabinet": "Metal or wood storage cabinet with lockable drawers for garage or workshop organization.",
    # Kitchenware & Dining
    "Cutlery Set": "16-piece, 24-piece, or 32-piece stainless steel cutlery set for everyday dining.",
    "Frying Pan": "Non-stick or cast iron frying pan in 8″, 10″, or 12″ diameter for versatile cooking.",
    "Dinnerware Set": "Porcelain or stoneware set for 4, 6, or 8, including plates, bowls, and mugs.",
    "Chef’s Knife": "High-carbon stainless steel chef’s knife in 6″, 8″, or 10″ lengths for precision slicing.",
    "Cutting Board": "Bamboo, plastic, or teak cutting board in small, medium, or large sizes.",
    "Mixer": "Hand mixer with multiple speed settings and attachments for whisking and mixing.",
    "Stand Mixer": "Heavy-duty stand mixer (e.g., 5 qt, 7 qt) with dough hook, whisk, and paddle attachments.",
    "Blender": "Countertop blender with high-speed motor and multiple blending presets for smoothies and soups.",
    "Food Processor": "7-cup or larger food processor with chopping, slicing, and shredding capabilities.",
    "Slow Cooker": "3 qt, 6 qt, or 8 qt programmable slow cooker for convenient, hands-off meal prep.",
    "Pressure Cooker": "Stovetop or electric pressure cooker with multiple pressure settings and safety locks.",
    "Coffee Grinder": "Burr or blade grinder for coarse to fine coffee grounds.",
    "Espresso Machine": "Semi-automatic or fully automatic espresso machine with PID temperature control.",
    "Toaster": "2-slice or 4-slice toaster with adjustable browning controls.",
    "Toaster Oven": "Compact toaster oven with convection and baking functions.",
    "Teapot": "Stainless steel or ceramic teapot for brewing loose leaf or bagged tea.",
    "Bakeware": "Non-stick cake pans, muffin tins, and baking sheets in standard US sizes.",
    "Glassware": "Wine glasses, tumblers, beer mugs, and shot glasses in sets of 4 or 6.",
    "Barware": "Cocktail shaker set with jigger, mixing spoon, and pour spouts for home bartending.",
    "Spice Rack": "Rotating or wall-mounted spice rack to store up to 20 jars of spices.",
    "Knife Block": "Wooden knife block or magnetic strip to organize and store kitchen knives safely.",
    "Dish Rack": "Stainless steel or plastic dish rack with utensil holder for countertop drying.",
    "Mortar & Pestle": "Granite or ceramic set for grinding herbs and spices by hand.",
    "Rolling Pin": "Traditional wooden or silicone rolling pin for baking pastries and dough.",
    "Measuring Cups": "Stainless steel or plastic sets for dry and liquid ingredient measurement.",
    "Measuring Spoons": "Nested sets of spoons in teaspoons and tablespoons for precise measurements.",
    "Dish Towels": "Cotton or microfiber towels for drying dishes and wiping countertops.",
    "Oven Mitts": "Heat-resistant mitts or gloves for safely handling hot cookware.",
    "Ice Maker": "Countertop ice maker producing bullet or nugget ice in minutes.",
    "Wine Cooler": "Dual-zone wine cooler for storing red and white wines at optimal temperatures.",
    "Water Dispenser": "Bottled or point-of-use water dispenser with hot and cold spouts.",
    "Portable BBQ Grill": "Charcoal or propane portable grill for tailgating, camping, or small patios.",
    # Beauty, Personal Care & Health
    "Face Cream": "Hydrating face cream infused with hyaluronic acid and antioxidants to nourish skin.",
    "Lipstick": "Long-lasting matte lipstick enriched with vitamins for rich color and hydration.",
    "Shampoo Bottle": "Sulfate-free shampoo in travel-size bottle that cleanses and moisturizes hair.",
    "Hair Serum": "Lightweight hair serum with argan oil to smooth frizz and add shine.",
    "Body Lotion": "Cocoa butter–based lotion providing deep moisture and a soft skin feel.",
    "Body Butter": "Shea butter–rich body butter for thick hydration and skin nourishment.",
    "Sunscreen": "Broad-spectrum SPF 30+ sunscreen to protect against UVA/UVB rays.",
    "Perfume": "Designer eau de parfum with top, heart, and base notes that linger all day.",
    "Makeup Palette": "Eyeshadow palette featuring matte, shimmer, and metallic finishes.",
    "Foundation": "Liquid or powder foundation offering buildable coverage and natural finish.",
    "Mascara": "Volumizing or lengthening mascara with waterproof formula for all-day wear.",
    "Eyeliner": "Gel, pencil, or liquid eyeliner for precise lines and long-lasting color.",
    "Nail Polish": "Gel or regular polish in assorted shades plus a top coat for chip resistance.",
    "Electric Toothbrush": "Sonic toothbrush with replaceable brush heads and multiple modes.",
    "Flosser": "Water flosser with adjustable pressure settings and tips for oral health.",
    "Electric Shaver": "Rotary or foil shaver with hypoallergenic blades for a smooth shave.",
    "Trimmer": "Rechargeable beard and nose trimmer with multiple guide combs.",
    "Hair Dryer": "Ionic hair dryer with ceramic technology and multiple speed/heat settings.",
    "Straightener": "Ceramic flat iron with adjustable temperature for sleek styling.",
    "Curling Iron": "Barrel curling iron in various diameters for loose waves or tight curls.",
    "Electric Razor": "Rechargeable electric razor with pop-up trimmer for grooming precision.",
    "Epilator": "Rotating tweezer epilator for long-lasting hair removal.",
    "Eyelash Curler": "Silicone pad eyelash curler for lifted, curled lashes.",
    "Hair Dryer Diffuser": "Diffuser attachment for curly hair to reduce frizz and define curls.",
    "Facial Cleansing Brush": "Silicone or bristle brush for deep pore cleaning and gentle exfoliation.",
    "Electric Massager": "Handheld massager with multiple heads for muscle relief and relaxation.",
    "Foot Spa": "Electric foot spa with heat and massage rollers for soothing foot care.",
    "First Aid Kit": "Comprehensive kit with bandages, antiseptics, gauze, and medical supplies.",
    "Vitamins": "Multivitamins, fish oil, and probiotics for daily health support.",
    "Supplements": "Herbal extracts, collagen peptides, and protein powders for wellness goals.",
    "Fitness Tracker": "Wearable tracker monitoring steps, heart rate, sleep patterns, and calories burned.",
    "Smart Scale": "Bluetooth-enabled scale that syncs weight, BMI, and body fat metrics to your phone.",
    "Yoga Mat": "Non-slip mat (68″×24″ or 72″×24″) made from eco-friendly materials for stability during workouts.",
    "Resistance Bands": "Set of latex or fabric resistance bands in varying tensions for strength training.",
    "Treadmill": "Foldable treadmill with digital console showing speed, distance, time, and calories burned.",
    "Exercise Bike": "Stationary upright or spin bike with adjustable resistance and digital display.",
    "Massage Gun": "High-torque percussive therapy device with interchangeable heads for muscle recovery.",
    "Yoga Blocks": "EVA foam blocks to support and deepen yoga poses, improving alignment and safety.",
    "Protein Powder": "Whey or plant-based protein powder available in flavors like chocolate, vanilla, and strawberry.",
    "Nutrition Bars": "Assorted granola, protein, and snack bars for on-the-go energy and nutrition.",
    "Blood Pressure Monitor": "Automatic cuff monitor for tracking systolic and diastolic pressure at home.",
    "Glucometer": "Blood glucose meter with test strips and lancets for diabetes management.",
    "Thermometer": "Digital thermometer for accurate body temperature readings within seconds.",
    "Pulse Oximeter": "Fingertip device to measure blood oxygen saturation (SpO₂) and pulse rate.",
    # Food & Beverage, Grocery & Gourmet
    "Organic Olive Oil": "Cold-pressed extra virgin olive oil sourced from organic farms, rich in antioxidants.",
    "Gourmet Chocolate": "Dark chocolate bars made with single-origin cocoa for an intense, rich flavor.",
    "Artisanal Cheese": "Handcrafted cheese aged to perfection, offering distinct textures and flavor profiles.",
    "Herbal Tea": "Organic tea blends like chamomile, peppermint, and hibiscus, promoting relaxation and wellness.",
    "Coffee Beans": "Specialty coffee beans available in light, medium, or dark roast, fresh-roasted for aroma.",
    "Ground Coffee": "Pre-ground coffee in various blends (espresso, drip, cold brew) for easy brewing.",
    "Spices": "Single spices (turmeric, cumin, paprika) sourced from global spice farms.",
    "Seasonings": "Blended mixes (garam masala, Cajun, Italian seasoning) to elevate your cooking.",
    "Organic Honey": "Raw, unfiltered honey in glass jars, retaining natural enzymes and antioxidants.",
    "Maple Syrup": "Grade A amber maple syrup harvested from sustainably managed maple trees.",
    "Snack Packs": "Gluten-free granola bars, trail mixes, and nut clusters for healthy snacking.",
    "Nutrition Bars": "Protein or granola bars fortified with vitamins, minerals, and fiber.",
    "Frozen Foods": "Frozen vegetables, fruits, and ready-to-heat meals like lasagna and dumplings.",
    "Vegan Products": "Plant-based meat alternatives like tofu, tempeh, and beyond meat burgers.",
    "Plant-Based Meat": "Impossible Burger or similar meat substitutes made from pea protein or soy.",
    "Health Foods": "Superfoods (chia seeds, goji berries) and supplements like collagen peptides.",
    "Supplements": "Collagen, probiotics, and herbal extracts in capsules or powders.",
    "Craft Beer": "Locally brewed craft beers in styles like IPA, stout, and lager.",
    "Wine": "Artisanal wines from vineyards around the world, available by varietal and region.",
    "Spirits": "Small-batch bourbon, Scotch, rum, vodka, and gin with tasting notes.",
    "Cold Pressed Juice": "Bottled cold-pressed juices made from fruits and vegetables without preservatives.",
    "Smoothie Packs": "Frozen fruit and vegetable packs pre-portioned for blending smoothies at home.",
    "Gourmet Snacks": "Truffle-flavored chips, artisanal popcorn (caramel, sea salt), and specialty nuts.",
    "Bakery Items": "Freshly baked bread, bagels, and pastries shipped overnight for ultimate freshness.",
    "Condiments": "Gourmet ketchup, mustard, mayo, and aioli in glass jars or squeeze bottles.",
    "Sauces": "Artisanal BBQ, hot sauces, and pasta sauces crafted in small batches.",
    "Meal Kits": "Pre-measured ingredients and step-by-step recipe cards for home-cooked meals.",
    "Pet Food": "Grain-free or organic dog and cat food made with real meat and vegetables.",
    # DIY & Hardware / Furniture & Lumber
    "Lawn Mower": "Cordless electric mower with adjustable cutting heights for easy lawn maintenance.",
    "Garden Hose": "Expandable, kink-resistant hose (50 ft or 100 ft) with a heavy-duty spray nozzle.",
    "Outdoor Furniture Set": "Weather-resistant rattan or teak table and chair set for patio relaxation.",
    "BBQ Grill": "Propane or charcoal grill with multiple burners and side shelf for outdoor cooking.",
    "Electric Drill": "Cordless drill with Li-ion battery, variable speed, and built-in LED work light.",
    "Impact Driver": "Cordless impact driver with high torque and quick-change chuck.",
    "Hammer": "Claw hammer with shock-absorbing handle for comfortable use.",
    "Crowbar": "Steel crowbar for prying and demolition tasks.",
    "Screwdriver Set": "Magnetic screwdrivers with multiple bits (flat, Phillips, Torx).",
    "Bit Kit": "Assorted drill and driver bit set for wood, metal, and masonry.",
    "Measuring Tape": "Retractable steel tape measure available in 16 ft and 25 ft lengths.",
    "Laser Distance Meter": "Digital laser distance meter for quick and precise measurements up to 100 ft.",
    "Socket Set": "Metric and SAE socket set with ¼″, ⅜″, and ½″ drive sizes.",
    "Wrench Set": "Combination wrench set in metric and SAE sizes made from chrome vanadium steel.",
    "Tool Chest": "Rolling tool chest with multiple drawers and lockable lid for workshop organization.",
    "Storage Cabinet": "Metal storage cabinet with adjustable shelves and lockable doors.",
    "Paint": "Interior and exterior latex paints in gallon, quart, and sample sizes.",
    "Paint Brushes": "Synthetic or natural bristle brushes in 1″, 2″, and 3″ widths.",
    "Power Saw": "Circular saw with adjustable bevel and rip fence for straight cuts.",
    "Jigsaw": "Orbital jigsaw with variable speed for intricate cuts in wood and metal.",
    "Nails": "Assorted nail kits (finishing, framing, roofing) in bulk packaging.",
    "Screws": "Wood screws, drywall screws, and masonry screws in multiple lengths and gauges.",
    "Lumber": "Dimensional lumber (2×4, 2×6, 2×8) in lengths from 8 ft to 16 ft.",
    "Plywood": "Sheets of plywood in ½″ and ¾″ thicknesses for furniture and construction.",
    "Safety Gear": "Work gloves, safety glasses, ear protection, and respirators for job site safety.",
    "Work Light": "Rechargeable LED work light with magnetic base and adjustable head.",
    "Flashlight": "Compact LED flashlight with zoom focus and multiple brightness modes.",
    "Extension Cord": "Heavy-duty, 12 AWG and 16 AWG extension cords in 25 ft and 50 ft lengths.",
    "Power Strip": "Surge-protected power strip with USB charging ports.",
    "Tool Belt": "Leather or nylon tool belt with multiple pouches and hammer loops.",
    "Apron": "Canvas or leather apron with pockets for tools and accessories.",
    "Paint Rollers": "9″ and 18″ roller covers and frames for painting large surfaces.",
    "Ladder": "Aluminum step stool (6 ft) and extension ladder (8 ft) for home and professional use.",
    # Office & School Supplies
    "Notebook": "Hardcover or spiral notebook in A5, A4 sizes with ruled, dotted, or blank pages.",
    "Journal": "Leather-bound journal with acid-free paper for writing or sketching.",
    "Pen Set": "Ballpoint, rollerball, and fountain pen set in an elegant gift box.",
    "Fountain Pen": "High-quality fountain pen with reusable ink cartridges and fine nib.",
    "Desk Organizer": "Multi-compartment desk organizer for pens, papers, and gadgets.",
    "File Holder": "Vertical or horizontal file holder made of metal or plastic for documents.",
    "Stapler": "Heavy-duty or compact desktop stapler with an anti-jam design.",
    "Staple Remover": "Ergonomic staple remover to easily extract staples without tearing paper.",
    "Printer": "All-in-one inkjet or laser printer with Wi-Fi connectivity and duplex printing.",
    "Scanner": "Flatbed scanner with high-resolution optical scanning and USB connectivity.",
    "Mouse Pad": "Extended or standard mouse pad with rubber base for stability.",
    "Wrist Rest": "Gel or memory foam wrist rest to support hands during extended typing.",
    "Desk Lamp": "LED desk lamp with touch controls, adjustable arm, and USB charging port.",
    "Task Light": "Clamp-on LED task light with flexible neck for focused illumination.",
    "Whiteboard": "Magnetic whiteboard in multiple sizes with mounting hardware included.",
    "Corkboard": "Cork bulletin board with pushpins for notes, photos, and reminders.",
    "Calculator": "Basic or scientific calculator for everyday math and educational use.",
    "Financial Calculator": "Advanced financial calculator for professionals and students.",
    "File Cabinet": "Metal or wood file cabinet with 2-drawer or 4-drawer configurations.",
    "Bookends": "Decorative or functional bookends to keep books upright on shelves.",
    "Binder": "3-ring binder in 1″, 2″, or 3″ sizes for organizing documents.",
    "Dividers": "Tabbed dividers for binders, labeled A–Z or by number for easy sorting.",
    "Backpack": "Padded backpack for laptops up to 17″ with multiple compartments.",
    "Laptop Bag": "Messenger or tote-style laptop bag with padded sleeve and shoulder strap.",
    "Markers": "Permanent, dry-erase, and flip-chart markers in assorted colors.",
    "Highlighters": "Fluorescent highlighters in pink, yellow, green, and blue for text emphasis.",
    "Paper": "A4 or A5 printer paper in 50-sheet reams or bulk packs.",
    "Envelopes": "Assorted-size envelopes for letters, invitations, and mailing needs.",
    "Sticky Notes": "Post-it notes in various sizes and neon colors for quick reminders.",
    "Index Tabs": "Repositionable tabs for marking pages in notebooks, binders, and books.",
    "Label Maker": "Handheld label printer with refillable tapes in multiple colors.",
    "Labels": "Label cartridges for label maker, assorted colors and widths.",
    "USB Flash Drive": "32 GB to 256 GB USB drives for portable data storage.",
    "External Hard Drive": "1 TB to 5 TB external HDD or SSD for backup and file transfers.",
    "Power Bank": "Portable power bank with multiple USB ports and fast-charge support.",
    "Charging Station": "Multi-port USB charging hub for phones, tablets, and accessories.",
    "Webcam": "1080p or 4K USB webcam with built-in microphone for video conferencing.",
    "Headset": "Noise-cancelling over-ear or in-ear headset with microphone for calls and gaming.",
    # Sports, Fitness & Outdoor
    "Yoga Mat": "Non-slip yoga mat (68″×24″, 72″×24″) made from eco-friendly TPE or PVC.",
    "Dumbbell Set": "Adjustable dumbbells ranging from 10 lb to 50 lb with secure locking collars.",
    "Kettlebell": "Cast-iron kettlebell in weights from 5 lb to 50 lb for strength training.",
    "Cycling Helmet": "Lightweight, ventilated helmet with adjustable straps for road cycling.",
    "Protective Gear": "Knee pads, elbow pads, and wrist guards for biking, skating, and scootering.",
    "Camping Tent": "Waterproof tent in 2-person, 4-person, or 6-person sizes with easy setup.",
    "Sleeping Bag": "Mummy or rectangular sleeping bags rated for 20°F to 40°F temperatures.",
    "Hiking Backpack": "30 L to 70 L backpacks with hydration bladder compatibility and multiple pockets.",
    "Trekking Poles": "Collapsible aluminum or carbon fiber poles with ergonomic grips.",
    "Fishing Rod": "Graphite or fiberglass spinning rod with adjustable reel seat.",
    "Reel": "Spinning reel with smooth drag system and corrosion-resistant materials.",
    "Golf Clubs": "Complete set including driver, irons (4–9), wedge, and putter with bag.",
    "Golf Bag": "Stand or cart bag with multiple pockets, waterproof base, and umbrella holder.",
    "Tennis Racket": "Graphite or composite racket with head sizes ranging from 90 sq in to 110 sq in.",
    "Tennis Balls": "Pressurized tennis balls in cans of three for professional or recreational play.",
    "Basketball": "Official size 7 basketball with durable TPU cover for indoor/outdoor use.",
    "Soccer Ball": "Size 5 soccer ball with TPU outer cover and machine-stitched panels.",
    "Camping Stove": "Portable gas stove with single or dual burners and wind guards.",
    "Cookware": "Nesting cookware sets (pot, pan, kettle) made from aluminum or stainless steel.",
    "Inflatable Kayak": "Single or tandem inflatable kayak with high-pressure floor and adjustable seats.",
    "Paddleboard": "Inflatable SUP board with non-slip deck pad and adjustable paddle.",
    "Outdoor Furniture": "Folding chairs, picnic tables, and portable seating for camping or backyard use.",
    "Hammock": "Woven or fabric hammock with tree straps for easy setup in yards or campsites.",
    "Pool Accessories": "Inflatable loungers, snorkel sets, goggles, and pool floats for leisure fun.",
    "Beach Accessories": "Beach umbrella, beach towel, cooler bag, and sand-free mat for seaside trips.",
    "Climbing Gear": "Climbing harness, carabiners, belay device, and helmet for rock climbing and bouldering.",
    "Backpacking Gear": "Ultralight tent, sleeping pad, and backpacking stove for long-distance hiking.",
    "Golf Apparel": "Moisture-wicking polo shirts, golf pants, and visors designed for golf courses.",
    "Golf Shoes": "Spiked or spikeless golf shoes with waterproof uppers and traction soles.",
    # Toys, Games & Collectibles
    "Action Figure": "Highly detailed, posable figures of superheroes, movie, or anime characters.",
    "Collectible Figurine": "Premium resin or PVC figurines in limited-edition releases for collectors.",
    "Board Game": "Strategy games like Catan, Ticket to Ride, or Cooperative games for family nights.",
    "Card Game": "Classic card games like UNO, Poker, or collectible card games like Pokémon and Magic: The Gathering.",
    "Puzzle": "500-piece or 1000-piece jigsaw puzzles featuring landscapes, artwork, or custom photos.",
    "Jigsaw Puzzle": "High-quality cardboard puzzles with intricate cut patterns for enthusiasts.",
    "Dollhouse": "Wooden or plastic dollhouses with miniature furniture and accessory sets.",
    "Miniature Sets": "Model kits for painting and assembling scale models (cars, planes, ships).",
    "Model Kit": "Plastic snap-together or glue-required kits for hobbyists and collectors.",
    "RC Vehicle": "Remote-controlled cars, trucks, boats, or drones with rechargeable batteries.",
    "Lego Set": "Official LEGO sets for ages 4–99, including themed collections like Star Wars and Technic.",
    "Building Blocks": "Non-branded building block sets for open-ended creativity and STEM learning.",
    "Stuffed Animal": "Plush toys including teddy bears, character plushies, and educational plush toys.",
    "Plush Toy": "Soft, machine-washable plush animals or characters for cuddling and comfort.",
    "Educational Toy": "STEM-based toys and kits teaching coding, circuitry, and robotics concepts.",
    "STEM Kit": "Robotics kits, circuitry boards, and sensor modules for hands-on STEM education.",
    "Video Game Console": "PlayStation 5, Xbox Series X, or Nintendo Switch game systems with bundles.",
    "Game Controller": "Wireless controllers, fight sticks, and steering wheels for immersive gaming.",
    "Arcade Stick": "Tournament-grade fight stick controllers for fighting games with arcade-style layout.",
    "VR Headset": "Standalone (Oculus Quest 2) or PC-tethered (Valve Index) headsets for virtual reality.",
    "Accessories": "Head straps, facial interfaces, and replacement VR lenses for headsets.",
    "Trackers": "External base stations or trackers for room-scale VR movement detection.",
    "Haptic Gloves": "Sensor-equipped gloves providing tactile feedback during VR experiences.",
    "Gaming Chair": "Ergonomic chair with adjustable lumbar support, built-in speakers, and RGB lighting.",
    "Organizer": "Toy bins, shelving units, or chests with labels for sorting and storing toys.",
    "Outdoor Play Equipment": "Backyard swing sets, slides, playhouses, and sandbox kits.",
    "Swing Set": "Metal or wooden swing sets with slide attachments and climbing gear.",
    "Drone": "Camera drones with GPS, return-to-home, and 4K video recording capabilities.",
    "RC Helicopter": "Miniature remote-controlled helicopters with gyroscopic stabilization.",
    # Automotive & Industrial
    "Car Cover": "Weather-resistant car cover protecting against dust, rain, and UV rays.",
    "Sunshade": "Foldable windshield sunshade to block heat and UV exposure inside the vehicle.",
    "Steering Wheel Cover": "Leather or fabric steering wheel cover for improved grip and style.",
    "Seat Covers": "Universal or custom-fit seat covers in leather, neoprene, or polyester.",
    "GPS Navigator": "Portable GPS device with real-time traffic updates and voice-guided directions.",
    "Dash Cam": "Front and rear dash camera system with loop recording and G-sensor detection.",
    "Car Vacuum": "12 V handheld vacuum cleaner with multiple attachments for interior cleaning.",
    "Portable Air Compressor": "Compact 12 V compressor with digital pressure gauge for inflating tires.",
    "Tire Inflator": "Cordless inflator with rechargeable battery and built-in pressure gauge.",
    "Jump Starter": "Portable jump starter with USB ports for charging phones and starting car batteries.",
    "Floor Mats": "All-weather rubber mats or carpeted mats designed to fit specific vehicle models.",
    "Trunk Organizer": "Collapsible trunk organizer with multiple compartments and removable dividers.",
    "Car Audio": "Aftermarket head units, subwoofers, and amplifiers for upgraded in-car sound systems.",
    "Speakers": "Component or coaxial speaker sets for front, rear, and door installations.",
    "Motor Oil": "Full-synthetic or conventional engine oil in SAE 0W-20, 5W-30 grades.",
    "Fluids": "Coolant, brake fluid, power steering fluid, and transmission fluid for maintenance.",
    "Brake Pads": "Ceramic, semi-metallic, or organic brake pads compatible with various vehicle models.",
    "Rotors": "Ventilated or drilled brake rotors for OEM replacement or performance upgrades.",
    "Car Battery": "Lead-acid or AGM batteries in Group sizes for passenger vehicles and trucks.",
    "Battery Charger": "Smart chargers with multiple charging modes and overcharge protection.",
    "Roof Rack": "Adjustable crossbars compatible with Thule or Yakima systems for carrying cargo.",
    "Cargo Carrier": "Rooftop cargo box or bag for extra storage on long trips and vacations.",
    "Tow Hitch": "Class III or IV hitch receiver kits with ball mounts and safety chains.",
    "Trailer Hitch": "5th-wheel or gooseneck hitch solutions for towing heavy trailers and RVs.",
    "Safety Hammer": "Emergency rescue tool featuring a seat belt cutter and window breaker tip.",
    "Seat Belt Cutter": "Compact, retractable blade tool for cutting seat belts in emergencies.",
    "Headlight Assembly": "OEM or aftermarket headlamp units with halogen, HID, or LED options.",
    "Taillight Assembly": "Complete taillight units compatible with various makes and models.",
    "Spark Plug": "Standard or iridium spark plugs for improved combustion and fuel efficiency.",
    "Ignition Coil": "Coil-on-plug or coil pack ignition coils for reliable engine performance.",
    "Air Filter": "High-flow engine air filters for improved airflow and horsepower.",
    "Cabin Filter": "Activated carbon or HEPA cabin air filters to trap pollutants and odors.",
    "Jumper Cables": "4-gauge or 6-gauge heavy-duty jumper cables for jump-starting vehicles.",
    "Tool Kit": "Basic automotive tool kit including wrenches, pliers, screwdrivers, and sockets.",
    "Industrial Lubricants": "Multipurpose grease and high-temperature industrial lubricants for machinery.",
    "Grease": "Premium automotive grease for chassis, wheel bearings, and universal joints.",
    "3D Printer": "Desktop FDM 3D printer with a heated bed and PLA/ABS compatibility.",
    "Filament": "PLA, ABS, or TPU filament spools in 1 kg sizes with various color options.",
    "CNC Accessories": "End mills, drill bits, and collets for CNC routers and milling machines.",
    "Router Bits": "High-speed steel or carbide bits for cutting, grooving, and shaping wood and plastic.",
    # Pet Supplies & Services
    "Dog Leash": "Retractable or standard dog leash with comfortable grip and reflective stitching.",
    "Collar": "Adjustable nylon or leather collar with metal buckle and ID tag ring.",
    "Cat Toy": "Feather teaser wands or interactive laser pointers to engage indoor cats.",
    "Laser Pointer": "Handheld laser pointer to stimulate cats’ natural hunting instincts.",
    "Pet Bed": "Orthopedic memory foam pet bed with removable, machine-washable cover.",
    "Crate": "Wire or plastic crate with removable tray and divider for growing puppies.",
    "Pet Food Bowl": "Stainless steel or ceramic bowls with non-slip rubber base for steady feeding.",
    "Feeder": "Automatic programmable feeder dispensing meals at scheduled times.",
    "Gourmet Pet Treats": "Grain-free, freeze-dried meat treats made with real protein sources.",
    "Grooming Kit": "Deshedding tool, nail clippers, slicker brush, and grooming scissors for pets.",
    "Aquarium": "Glass or acrylic fish tank (10 gal, 20 gal, 50 gal) with LED lighting and filter.",
    "Fish Tank": "Complete aquarium kits with hood, filter, and LED lighting included.",
    "Bird Cage": "Wire cage with wooden perches, feeding bowls, and swing toys for birds.",
    "Perch": "Natural wood or plastic perches to place in bird cages for roosting.",
    "Small Animal Hutch": "Wooden or wire hutch for rabbits, guinea pigs, or hamsters with removable tray.",
    "Cage": "Plastic-bottom wire cages for small pets with multiple levels and ramps.",
    "Pet Carrier": "Airline-approved soft or hard-sided carrier for dogs and cats up to 20 lb.",
    "Travel Crate": "Ventilated, collapsible travel crate for safe pet transport by car or plane.",
    "Flea Preventive": "Topical or oral treatments to protect pets from fleas and ticks.",
    "Tick Preventive": "Monthly oral chews or collars to prevent tick infestations.",
    "Dewormer": "Oral deworming tablets to treat common intestinal parasites in pets.",
    "Pet Clothing": "Pet sweaters, raincoats, and booties designed for comfort and warmth.",
    "Accessories": "Collars, bandanas, bow ties, and harnesses for styling pets.",
    "Pet ID Tag": "Engraved metal ID tag with pet name and owner contact information.",
    "Microchip Service": "Pet microchipping kits and professional implantation for permanent ID.",
    "Pet Insurance": "Monthly or annual insurance plans covering accidents and illnesses.",
    "Veterinary Plan": "Subscription-based wellness plan covering routine checkups and vaccines.",
    "Training Classes": "Online or in-person obedience, agility, and behavioral training courses.",
    "Behavior Consultation": "One-on-one sessions with certified behaviorists for training support.",
    # Books, Media & Digital Content
    "Paperback": "Pocket-size or trade paperback novels, non-fiction, and academic titles.",
    "Hardcover": "Durable hardcover editions with dust jackets for premium book collections.",
    "Audiobook": "Digital audiobooks narrated by professionals, available via subscription or purchase.",
    "E-Book": "Kindle or EPUB files for novels, self-published works, and technical manuals.",
    "Digital Download": "MP3, PDF, or ZIP files containing music, software, or e-books.",
    "Magazine Subscription": "Print or digital subscriptions to lifestyle, trade, or academic magazines.",
    "Journal Subscription": "Subscription to peer-reviewed academic journals in various disciplines.",
    "Music Album": "CD or vinyl LPs featuring full-length albums in pop, rock, jazz, and classical genres.",
    "Vinyl Record": "Collector-grade vinyl records in 33⅓ RPM or 45 RPM formats for audiophiles.",
    "Blu-ray": "High-definition Blu-ray discs for movies, TV series, and special features.",
    "DVD": "Standard-definition DVD releases of films, series, and documentaries.",
    "4K UHD": "Ultra HD Blu-ray discs with HDR support for the ultimate home theater experience.",
    "Streaming Service Subscription": "Monthly or yearly plans for Netflix, Disney+, and Hulu.",
    "Video Game Title": "Physical or digital copies of console/PC games across all genres.",
    "Digital DLC": "Downloadable content packs expanding existing video games with new levels or skins.",
    "Software License": "Single-user or multi-user licenses for desktop or enterprise software.",
    "Productivity Suite": "Office software bundles like Microsoft 365 or LibreOffice for documents and spreadsheets.",
    "Cloud Storage Subscription": "Plans for Dropbox, Google Drive, or OneDrive in 100 GB, 1 TB, or 5 TB tiers.",
    "Online Course": "Udemy, Coursera, or LinkedIn Learning courses in coding, design, business, and more.",
    "e-Learning Platform": "Subscriptions to platforms offering certificates and professional development tracks.",
    "Virtual Conference Ticket": "Access passes to live-streamed industry conferences with networking features.",
    "Webinar Pass": "Individual or group passes for specialized webinars in tech, finance, or marketing.",
    "Stock Photos": "Royalty-free image packs and collections for designers and marketers.",
    "Graphics Pack": "Sets of icons, templates, and vector graphics for creative projects.",
    "NFT": "Blockchain-based digital artworks, virtual real estate, or collectible cards.",
    "Digital Collectible": "Unique in-game items, skins, or tokens redeemed via blockchain or centralized platforms.",
    # Art Supplies & Crafting
    "Canvas Set": "Pre-stretched canvases in sizes 8×10, 12×12, and 16×20 for painting projects.",
    "Easel": "Adjustable wooden or metal easels for studio or tabletop use.",
    "Paint Brushes": "Synthetic or natural bristle brushes in sets of 5, 10, or 20 for acrylic and oil painting.",
    "Palette Knives": "Stainless steel or carbon steel knives for mixing and applying paint textures.",
    "Acrylic Paint": "Tube or student-grade acrylic paints in sets of 12, 24, or 48 colors.",
    "Oil Paint": "Artist-grade oil paints in tubes, available individually or in sets of primary colors.",
    "Watercolor Paint": "Pans or tubes of watercolor paint in sets of 12, 24, or 48 hues.",
    "Sketchbook": "Hardcover sketchbook with acid-free paper in A4 or A3 sizes.",
    "Drawing Pad": "Spiral-bound drawing pad with heavyweight paper for pencil, charcoal, or ink.",
    "Watercolor Palette": "Portable metal palette containing 24 or 48 watercolor pans and mixing tray.",
    "Colored Pencils": "Prismacolor or Faber-Castell colored pencil sets ranging from 12 to 72 pencils.",
    "Markers": "Copic, Prismacolor, or Sharpie marker sets in a variety of tips and colors.",
    "Crafting Kit": "DIY kits for knitting, embroidery, soap making, or candle making with all materials included.",
    "DIY Project Box": "Subscription boxes delivering supplies and instructions for monthly crafting projects.",
    "Sewing Machine": "Portable sewing machines with presser feet kits, bobbins, and basic stitch options.",
    "Accessories": "Presser feet, bobbins, needles, and thread kits for sewing machine users.",
    "Scrapbooking Supplies": "Patterned papers, decorative stickers, washi tape, and adhesives for memory books.",
    "Stickers": "Vinyl or paper stickers in assorted themes for planners, scrapbooks, or laptops.",
    "Beads": "Glass, acrylic, or metal beads with elastic cord and jewelry-making tools included.",
    "Jewelry Making Supplies": "Wire, pliers, clasps, and charms for creating custom necklaces, bracelets, and earrings.",
    "Pottery Tools": "Clay sculpting tools, wire cutters, and pottery wheel accessories for ceramics.",
    "Clay": "Air-dry clay, polymer clay, and stoneware clay for sculpting and pottery projects.",
    "3D Printer": "Desktop FDM 3D printer with heated bed, compatible with PLA, ABS, and TPU filaments.",
    "Filament": "1 kg PLA, ABS, or TPU filament spools in a spectrum of colors for 3D printing.",
    "Digital Drawing Tablet": "Wacom or Huion tablets with pressure-sensitive stylus pens and customizable buttons.",
    "Stylus": "Bluetooth-enabled stylus pens compatible with tablets and touchscreen devices for drawing.",
    "Studio Lighting": "LED panel lights, softbox kits, and ring lights for photography and video shoots.",
    # Baby & Toddler
    "Baby Stroller": "Lightweight stroller with one-hand fold system and multi-position recline.",
    "Travel System": "Combo with infant car seat and stroller that snaps together for mobility.",
    "Diapers": "Disposable or cloth diapers available in sizes Newborn through Size 6.",
    "Pull-Ups": "Training pants with elastic waist for toddlers transitioning out of diapers.",
    "Baby Monitor": "Wi-Fi–enabled video monitor with night vision and two-way audio.",
    "Video Monitor": "Analog or digital monitor with camera and parent unit for baby surveillance.",
    "Baby Lotion": "Hypoallergenic, fragrance-free lotion formulated for sensitive baby skin.",
    "Baby Shampoo": "Tear-free, gentle shampoo with natural ingredients like aloe vera and chamomile.",
    "Crib": "Convertible crib with adjustable mattress heights and teething rails.",
    "Bassinet": "Portable bassinet with mesh sides and a rocking base for newborns.",
    "Changing Table": "Wooden or plastic changing table with safety straps and storage shelves below.",
    "Changing Pad": "Waterproof changing pad insert with contoured sides for baby safety.",
    "Baby Carrier": "Ergonomic baby carrier for front, back, and hip carry in cotton or mesh fabrics.",
    "Wrap": "Long fabric wrap for newborn snuggles and hands-free carrying.",
    "Sling": "Ring sling made from lightweight fabric for quick and easy baby wearing.",
    "High Chair": "Adjustable high chair with removable tray and washable seat cover.",
    "Booster Seat": "Portable booster cushion with safety straps for toddlers at the table.",
    "Playard": "Foldable play yard with bassinet attachment, changing station, and storage pockets.",
    "Pack & Play": "Multi-functional pack and play with changing table and bassinet insert included.",
    "Toddler Toy": "Developmental toys like shape sorters, stacking rings, and activity cubes.",
    "Educational Toy": "STEM-based toys such as building blocks, puzzles, and interactive kits.",
    "Baby Clothes": "Onesies, rompers, sleepers in 0–3, 3–6, and 6–12 month sizes made from cotton.",
    "Bodysuits": "Snap-crotch bodysuits in short- and long-sleeve varieties for easy diaper changes.",
    "Pacifier": "BPA-free pacifiers with orthodontic silicone nipples for soothing baby.",
    "Teethers": "Silicone or rubber teething rings designed to soothe sore gums.",
    "Car Seat": "Rear-facing, convertible car seats with LATCH installation and adjustable harness.",
    "Booster Seat": "Forward-facing booster seat with cup holders and adjustable headrest for toddlers.",
    "Baby Bottle": "Anti-colic bottles with slow-flow nipples to reduce gas and spit-up.",
    "Sterilizer": "Electric steam sterilizer for baby bottles, pacifiers, and small accessories.",
    "Breast Pump": "Electric double breast pump with adjustable suction levels and closed system for hygiene.",
    "Nursing Pillow": "U-shaped pillow providing support and comfort during breastfeeding sessions.",
    "Diaper Bag": "Nylon or leather diaper bag with insulated bottle pockets, changing mat, and multiple compartments.",
    "Backpack": "Hands-free diaper backpack with stroller straps, insulated pockets, and wet/dry compartments.",
    "Changing Mat": "Portable, foldable changing mat with waterproof lining for on-the-go diaper changes.",
    "Baby Blanket": "Muslin or fleece swaddle blankets in assorted prints for warmth and comfort.",
    "Swaddle": "Stretchy cotton or bamboo wraps for snugly swaddling newborns to mimic womb environment.",
    "Baby Shoes": "Soft-soled leather or knit booties for pre-walkers to protect tiny feet.",
    "Booties": "Knit or fleece booties with elastic ankles to keep baby’s feet warm.",
    "Safety Gate": "Pressure-mounted gate for doorways and staircases to childproof homes.",
    "Corner Guards": "Foam or silicone corner protectors to cushion sharp furniture edges.",
    # Jewelry & Watches
    "Necklace": "Gold-plated, sterling silver, or gemstone necklace with adjustable chain lengths.",
    "Pendant": "Decorative pendant charms in metals like gold, silver, and stainless steel.",
    "Earrings": "Sterling silver, gold, or stainless steel earrings—studs, hoops, or dangles.",
    "Studs": "Minimalist gemstone or metal studs for everyday wear.",
    "Hoops": "Classic hoop earrings in small, medium, and large diameters.",
    "Bracelet": "Charm bracelets, cuff bracelets, or chain bracelets in various metals.",
    "Bangles": "Solid or split bangles in gold, silver, or mixed-metal finishes.",
    "Ring": "Solitaire, halo, or stackable rings in precious metals with gemstone accents.",
    "Band": "Wedding or fashion bands in platinum, gold, titanium, or tungsten.",
    "Anklet": "Sterling silver or beaded anklet for a subtle foot accessory.",
    "Toe Ring": "Simple metal toe rings in sterling silver or gold-plated finishes.",
    "Brooch": "Enamel or gemstone brooches for lapels, scarves, or hats.",
    "Lapel Pin": "Metal lapel pins featuring logos, flags, or enamel designs for suits and jackets.",
    "Cufflinks": "Gold, silver, or novelty cufflinks to add flair to French-cuff shirts.",
    "Tie Clip": "Matching tie clips in metal and enamel to keep ties neatly in place.",
    "Jewelry Organizer": "Velvet-lined or leather jewelry box with compartments for rings, earrings, and necklaces.",
    "Watch Winder": "Automatic winding box for mechanical watches with programmable rotation settings.",
    "Watch Case": "Travel watch case with padded compartments to protect watches on the go.",
    # Party & Festivity
    "Party Lights": "LED string lights, fairy lights, or marquee letters in various lengths and colors.",
    "String Lights": "Energy-efficient string lights for indoor and outdoor decor, battery-operated or plug-in.",
    "Inflatable Decorations": "Seasonal yard inflatables like Santa, snowmen, pumpkins, or Easter bunnies.",
    "Yard Inflatables": "Large inflatable figures for holidays with built-in blowers and stakes.",
    "Birthday Banner": "Customizable banners made of cardstock or vinyl in various fonts and colors.",
    "Custom Banner": "Vinyl or fabric banners printed with personalized messages for events.",
    "Confetti Pack": "Biodegradable confetti in tissue or paper, assorted shapes and colors.",
    "Tableware": "Themed disposable plates, cups, napkins, and cutlery for parties and events.",
    "Disposable Plates": "Paper or plastic plates in standard (10″) and small (7″) sizes.",
    "Helium Tank": "Disposable helium tank for inflating up to 50 latex or foil balloons.",
    "Balloons": "Latex or foil balloons in numbers, letters, and shapes with helium or air inflation.",
    "Photo Booth Props": "Cardstock props—hats, glasses, mustaches—for fun photo backdrops.",
    "Backdrops": "Fabric, vinyl, or paper backdrops with printed designs for photo booths.",
    "Party Favors": "Small toys, candies, and novelty items packaged in themed favor bags.",
    "Goodie Bags": "Pre-packaged treat bags for kids’ parties filled with toys and snacks.",
    "Cake Topper": "Acrylic or wooden cake toppers with custom text or designs.",
    "Dessert Stand": "Multi-tiered stands for displaying cupcakes, cake pops, and pastries.",
    "Sound System": "Portable PA speakers, microphones, and mixers for DJ or live events.",
    "DJ Equipment": "DJ controller, mixer, turntables, and lighting effects for party setups.",
    "Event Tickets": "Printed or e-tickets for concerts, sports games, theaters, and festivals.",
    "Passes": "Wristbands or badge passes for conferences, trade shows, and VIP events.",
    # Video Games & Virtual Reality
    "Gaming Console": "PlayStation 5, Xbox Series X, or Nintendo Switch consoles with bundle options.",
    "Hardware": "Performance gaming PCs, CPU, GPU, RAM, and storage components for custom builds.",
    "Game Controller": "Wireless controllers, arcade sticks, racing wheels, and pedal sets for immersion.",
    "Arcade Stick": "Tournament-grade fight stick controllers with arcade-style joystick and buttons.",
    "Racing Wheel": "Force-feedback steering wheels with pedal sets for racing simulators.",
    "Pedals": "Adjustable throttle, brake, and clutch pedals compatible with sim rigs.",
    "VR Headset": "Standalone (Oculus Quest 2) or PC-tethered (Valve Index, HTC Vive) VR headsets.",
    "Accessories": "Head straps, facial interfaces, prescription lens inserts, and replacement face pads.",
    "Trackers": "External base stations or SteamVR trackers for full-room VR movement detection.",
    "Haptic Gloves": "Sensor-equipped gloves providing tactile feedback during VR interactions.",
    "Gaming Chair": "Ergonomic chairs with lumbar support, built-in speakers, and RGB lighting effects.",
    "Racing Simulator": "Cockpit frames with mounting points for wheel, pedals, and seat for racing setups.",
    "Gaming Headset": "7.1 surround-sound headsets with noise-cancelling microphones and LED accents.",
    "Microphone": "USB or XLR microphones for streaming and podcasting with pop filters and stands.",
    "Monitor": "High-refresh-rate (144 Hz, 240 Hz) monitors in 1080p, 1440p, and 4K resolutions.",
    "Gaming Monitor": "G-Sync or FreeSync gaming monitors with 1 ms response times and low input lag.",
    "Graphics Card": "NVIDIA RTX 30/40 series or AMD Radeon RX 6000/7000 series GPUs for high performance.",
    "GPU": "Discrete graphics cards supporting ray tracing and AI-enhanced upscaling (DLSS, FSR).",
    "Gaming PC": "Prebuilt desktop gaming rigs with custom liquid cooling, RGB lighting, and overclocked CPUs.",
    "Components": "Individual parts like motherboards, RAM, cooling systems, and power supplies for DIY builds.",
    "Gaming Laptop": "Portable gaming laptops featuring high-end GPUs, 144 Hz+ displays, and advanced cooling.",
    "Portable Gaming": "Handheld gaming devices like Steam Deck or Nvidia Shield for on-the-go play.",
    "Streaming Gear": "Elgato capture cards, green screens, ring lights, and stream decks for content creation.",
    "Capture Card": "USB or PCIe capture cards supporting up to 4K passthrough and recording resolutions.",
    "Green Screen": "Collapsible or wall-mounted green screens for chroma key backgrounds in streams.",
    "Ring Light": "Adjustable ring lights with CRI 95+ for even, flattering lighting during webcam sessions.",
    "Game Title": "Physical or digital copies of AAA, indie, or retro games for console and PC.",
    "Digital DLC": "Downloadable content packs that add new maps, characters, skins, or storylines to games.",
    "Game Subscription": "Monthly services like Xbox Game Pass, PlayStation Plus, and Nintendo Switch Online.",
    "Online Service": "Online game stores and subscription services providing access to large game libraries.",
    "In-Game Currency": "V-bucks, gold, points, or credits used for purchasing cosmetics, skins, or items in games.",
    # Software, Subscriptions & Digital Services
    "Antivirus Software": "Norton, McAfee, or Bitdefender annual subscriptions protecting multiple devices from malware.",
    "Security Software": "Comprehensive suites with firewall, VPN, password manager, and identity theft protection.",
    "Photo Editing Software": "Adobe Photoshop, Lightroom, or Affinity Photo licenses for professional editing.",
    "Video Editing Software": "Adobe Premiere Pro, Final Cut Pro, or DaVinci Resolve with advanced editing features.",
    "Office Suite": "Microsoft 365, Google Workspace, or LibreOffice bundles for documents, spreadsheets, and presentations.",
    "Productivity Tools": "Note-taking apps (Evernote), task managers (Todoist), and calendar apps (Calendly).",
    "Cloud Storage": "Dropbox, Google Drive, OneDrive, or iCloud plans in 100 GB, 1 TB, or 5 TB tiers.",
    "Backup Service": "Backblaze, Carbonite, or CrashPlan subscription for automatic cloud backups.",
    "Website Hosting": "Shared hosting, VPS, or dedicated servers from providers like Bluehost, SiteGround, and DigitalOcean.",
    "Domain Service": "Domain registration for .com, .net, .org, and country-specific TLDs via registrars like GoDaddy.",
    "VPN Subscription": "NordVPN, ExpressVPN, or Surfshark monthly or yearly plans with multi-device support.",
    "Streaming Service": "Netflix, Disney+, HBO Max, Hulu, and Amazon Prime Video plans with HD/4K streaming.",
    "Digital Marketing Tool": "Ahrefs, SEMrush, Moz Pro, or HubSpot subscriptions for SEO, analytics, and marketing automation.",
    "SEO Software": "Keyword research, backlink tracking, and rank monitoring platforms to improve website visibility.",
    "Project Management Software": "Asana, Trello, Monday.com, or Jira subscriptions for team collaboration and workflows.",
    "E-Book Subscription": "Kindle Unlimited or Scribd monthly plans providing access to a vast library of e-books.",
    "Audiobook Service": "Audible memberships offering monthly credits to purchase audiobooks and podcasts.",
    "VPS": "Virtual private servers from AWS EC2, DigitalOcean, Linode, or Google Cloud with scalable pricing.",
    "Cloud Compute": "Pay-as-you-go compute instances on AWS, Azure, or Google Cloud Platform for apps and services.",
    "SaaS Tool": "Business software like Salesforce CRM, Slack, Zoom, and Atlassian Jira subscriptions for teams.",
    # Eco-Friendly & Sustainable Products
    "Reusable Water Bottle": "Insulated stainless steel or glass bottle designed to keep beverages hot or cold.",
    "Tumblers": "Reusable tumblers with straw lids made from BPA-free plastic or stainless steel.",
    "Biodegradable Straws": "Compostable paper or PLA straws that break down quickly in compost bins.",
    "Cutlery": "Bamboo or wooden cutlery sets—forks, knives, spoons—that are eco-friendly and compostable.",
    "Reusable Grocery Bags": "Organic cotton, canvas, or recycled PET bags to replace single-use plastic bags.",
    "Produce Bags": "Mesh or cotton drawstring bags for fruits and vegetables, washable and durable.",
    "Solar Charger": "Foldable solar panel charger for USB devices, ideal for camping and outdoor use.",
    "Solar Panel": "Portable solar panels with USB or DC output for off-grid power generation.",
    "Bamboo Toothbrush": "Toothbrushes with bamboo handles and biodegradable bristles to reduce plastic waste.",
    "Dental Floss": "Silk or PLA-based dental floss in compostable packaging for zero-waste oral care.",
    "Beeswax Wraps": "Reusable food wraps made from cotton coated in beeswax, jojoba oil, and resin.",
    "Reusable Food Wraps": "Silicone or fabric wraps that seal around containers and produce to preserve freshness.",
    "Eco-Friendly Cleaning Products": "Plant-based detergents, refillable cleaning concentrate bundles to reduce plastic.",
    "Compost Bin": "Kitchen countertop bins or outdoor tumblers for composting food scraps and yard waste.",
    "Compost Tumbler": "Rotating compost tumbler that speeds up decomposition of organic matter.",
    "Organic Cotton Towels": "GOTS-certified cotton towels in sets for bath and kitchen use.",
    "Bedsheets": "Organic cotton or bamboo fiber bedsheets in Queen, King, and California King sizes.",
    "Sustainable Fashion": "Apparel made from organic, recycled, or ethically sourced materials.",
    "Ethical Apparel": "Fair-trade certified clothing brands ensuring safe labor practices and eco-friendly production.",
    "Plantable Stationery": "Notebooks and greeting cards embedded with seeds that sprout when planted in soil.",
    # Luxury Goods & Premium Segments
    "Luxury Handbag": "Designer handbags from Louis Vuitton, Gucci, or Prada crafted from premium leather.",
    "Designer Purse": "High-end purses featuring iconic logos, craftsmanship, and limited-edition designs.",
    "High-End Watch": "Swiss timepieces from Rolex, Patek Philippe, or Omega made with precious metals.",
    "Swiss Timepiece": "Mechanical watches with intricate movements, often featuring complications like moonphase.",
    "Premium Spirit": "Single malt Scotch, small-batch bourbon, or artisanal gin aged in oak barrels.",
    "Aged Whiskey": "Premium bourbon or Scotch aged 12+ years, offering complex flavor notes of oak and vanilla.",
    "Designer Clothing": "Runway pieces from Dior, Chanel, Prada, or Balenciaga featuring exceptional tailoring.",
    "Haute Couture": "Bespoke fashion garments crafted by couture houses with hand-sewn embellishments.",
    "Fine Jewelry": "Platinum or 18 K gold jewelry set with diamonds, emeralds, sapphires, or rubies.",
    "Gemstone Ring": "Solitaire or halo rings featuring high-quality diamonds or colored gemstones.",
    "Collector Car": "Vintage or limited-production sports cars such as a 1960s Ferrari or classic Porsche.",
    "Luxury Automobile": "High-performance vehicles like Ferrari, Lamborghini, Rolls-Royce, or Bentley.",
    "Art & Antique": "Original paintings, sculptures, or collectible antiques authenticated by experts.",
    "Limited Edition Print": "Signed and numbered art prints or lithographs by renowned artists.",
    "Yacht": "Motor yachts or sailing yachts equipped with luxury cabins, engines, and navigation systems.",
    "Private Jet": "Light, mid-size, or heavy jets equipped with private cabins for luxury air travel.",
    "Custom-Built Home": "Architect-designed residences built with premium materials, smart home integration, and bespoke finishes.",
    # Recommerce & Pre-Owned / Refurbished
    "Refurbished Electronic": "Renewed smartphones, laptops, or tablets restored to like-new condition with warranty.",
    "Certified Pre-Owned Device": "Manufacturer-backed pre-owned devices (phones, laptops) inspected and certified.",
    "Pre-Owned Luxury Bag": "Authentic designer handbags sold gently used, authenticated by experts.",
    "Designer Apparel": "Gently used designer clothing items from high-end brands verified for authenticity.",
    "Used Book": "Secondhand or out-of-print books sold through online marketplaces like AbeBooks or eBay.",
    "Collectible Edition": "Limited-run or rare editions of books, comics, or vinyl records for collectors.",
    "Vintage Vinyl Record": "Original pressed vinyl records from classic albums in good condition.",
    "Secondhand Instrument": "Pre-owned guitars, keyboards, or other musical instruments sold at a discount.",
    # Healthcare & Medical Supplies
    "Medical Mask": "Certified N95 or KN95 respirator masks providing airborne particle protection.",
    "Respirator": "Medical-grade respirators with adjustable nose clips and multiple layer filters.",
    "Hand Sanitizer": "Alcohol-based (≥60%) sanitizers in pump bottles or travel-sized containers.",
    "Disinfectant Wipes": "Multi-surface disinfectant wipes effective against viruses and bacteria.",
    "First Aid Kit": "Home or office first aid kits containing bandages, antiseptics, gauze, and medical tools.",
    "Blood Pressure Monitor": "Digital arm cuff monitors for at-home tracking of blood pressure readings.",
    "Glucometer": "Blood glucose meter kits with test strips and lancets for diabetes management.",
    "Wheelchair": "Folding or electric wheelchairs with adjustable footrests and cushioned seats.",
    "Mobility Aid": "Walkers, rollators, and canes for individuals needing assistance with mobility.",
    "CPAP Machine": "Continuous positive airway pressure device with humidifier for sleep apnea therapy.",
    "Sleep Apnea Device": "Auto-adjusting CPAP machines with pressure relief and mask options.",
    "Diagnostic Test Kit": "At-home rapid test kits for COVID-19, influenza, strep throat, etc.",
    "Orthopedic Support": "Knee braces, wrist supports, ankle braces for injury prevention and recovery.",
    "Braces": "Medical-grade braces for knee, elbow, or back support during rehabilitation.",
    "Prescription Medication": "Refillable prescriptions delivered via online pharmacies for chronic conditions.",
    "Telemedicine Service": "Virtual doctor consultations and prescription services via secure telehealth platforms.",
    # Subscriptions & Memberships
    "Meal Kit Delivery": "Weekly meal kit subscriptions (Blue Apron, HelloFresh) with pre-measured ingredients.",
    "Prepared Meal Plan": "Ready-to-eat meal subscriptions tailored to dietary preferences (e.g., keto, vegan).",
    "Streaming Video": "Monthly or yearly plans for Netflix, Disney+, HBO Max, or Hulu in HD/4K.",
    "Streaming Music": "Spotify Premium, Apple Music, Tidal HiFi subscriptions for ad-free music streaming.",
    "Subscription Box": "Birchbox (beauty), FabFitFun (lifestyle), Stitch Fix (personal styling), BarkBox (pet treats).",
    "Gym Membership": "Local gym, CrossFit, or boutique fitness studio memberships with monthly or annual options.",
    "Digital Publication": "Digital newspaper or magazine subscriptions to NYT, WSJ, The Economist.",
    "Professional Network": "LinkedIn Premium, MasterClass, Coursera Plus, or Skillshare monthly/annual plans.",
    "Gaming Subscription": "Xbox Game Pass, PlayStation Plus, EA Play, Nintendo Switch Online memberships.",
    "Online Service": "SaaS tools like Jira, Asana, Slack, or Salesforce with tiered payment options.",
    }
  
    color = random.choice([
    "Red", "Blue", "Black", "White", "Green", "Beige", "Ivory", "Lavender", "Fuchsia", "Carmine",
    "Viridian", "Mauve", "Baby Blue", "Emerald", "Turquoise", "Amber", "Coral", "Peach", "Magenta", "Crimson",
    "Aqua", "Indigo", "Olive", "Charcoal", "Maroon", "Sapphire", "Plum", "Teal", "Rose", "Lime",
    "Mustard", "Slate", "Copper", "Jade", "Periwinkle", "Cobalt", "Blush", "Mint", "Sunset Orange", "Seafoam"])
    
    product_sizes = {
    # Apparel & Accessories
    "Shirt": ["XS", "S", "M", "L", "XL", "XXL"],
    "Shoes": ["6", "7", "8", "9", "10", "11", "12"],
    "Watch": ["38mm", "40mm", "42mm", "44mm"],
    "Jacket": ["S", "M", "L", "XL", "XXL"],
    "Backpack": ["Small", "Medium", "Large"],
    "Sunglasses": ["One Size"],
    "Sweater": ["S", "M", "L", "XL"],
    "Kurta": ["S", "M", "L", "XL", "XXL"],
    "Jeans": ["28", "30", "32", "34", "36", "38"],
    "T-shirt": ["XS", "S", "M", "L", "XL", "XXL"],
    "Scarf": ["One Size"],
    "Hat": ["S", "M", "L"],
    "Sneakers": ["6", "7", "8", "9", "10", "11", "12"],
    "Boots": ["6", "7", "8", "9", "10", "11", "12"],
    "Blazer": ["S", "M", "L", "XL"],
    "Wallet": ["One Size"],
    "Gloves": ["S", "M", "L"],
    "Belt": ["28", "30", "32", "34", "36", "38", "40"],
    "Tie": ["One Size"],
    "Cufflinks": ["One Size"],
    "Chinos": ["28", "30", "32", "34", "36", "38"],
    "Cardigan": ["S", "M", "L", "XL"],
    "Pajamas": ["S", "M", "L", "XL"],
    "Shorts": ["S", "M", "L", "XL"],
    "Raincoat": ["S", "M", "L", "XL"],
    "Sweatshirt": ["S", "M", "L", "XL"],
    "Tracksuit": ["S", "M", "L", "XL"],
    "Vest": ["S", "M", "L", "XL"],
    "Overcoat": ["S", "M", "L", "XL"],
    "Sandals": ["6", "7", "8", "9", "10", "11"],
    "Flip-flops": ["6", "7", "8", "9", "10", "11"],
    "Beanie": ["One Size"],
    "Headband": ["One Size"],
    "Socks": ["S", "M", "L"],
    "Umbrella": ["Small", "Medium", "Large"],
    "Luggage": ["20\"", "24\"", "28\""],
    "Duffel Bag": ["Small", "Medium", "Large"],
    "Laptop Sleeve": ["13\"", "14\"", "15.6\""],
    # Electronics & Appliances
    "Smartphone": ["5.5\"", "6.1\"", "6.7\""],
    "Laptop": ["13\"", "14\"", "15.6\"", "17.3\""],
    "Tablet": ["7\"", "8\"", "10.2\"", "12.9\""],
    "Smartwatch": ["38mm", "40mm", "44mm", "46mm"],
    "Bluetooth Headphones": ["One Size"],
    "Wireless Earbuds": ["One Size"],
    "Smart TV": ["32\"", "43\"", "50\"", "55\"", "65\"", "75\"", "85\""],
    "Digital Camera": ["APS-C", "Full Frame"],
    "Portable Speaker": ["Small", "Medium", "Large"],
    "Power Bank": ["5,000mAh", "10,000mAh", "20,000mAh"],
    "Electric Kettle": ["1L", "1.5L", "2L"],
    "Hair Dryer": ["One Size"],
    "Electric Toothbrush": ["One Size"],
    "Electric Shaver": ["One Size"],
    "Microwave Oven": ["20L", "25L", "30L", "32L", "35L"],
    "Refrigerator": ["200L", "300L", "500L"],
    "Washing Machine": ["6kg", "8kg", "10kg"],
    "Air Conditioner": ["1 Ton", "1.5 Ton", "2 Ton"],
    "Water Purifier": ["5L", "7L", "10L"],
    "Electric Fan": ["One Size"],
    "Induction Cooktop": ["One Size"],
    "Electric Pressure Cooker": ["One Size"],
    "Electric Grill": ["One Size"],
    "Electric Rice Cooker": ["One Size"],
    "Electric Juicer": ["One Size"],
    "Electric Mixer Grinder": ["One Size"],
    "Electric Iron": ["One Size"],
    "Electric Heater": ["One Size"],
    "Electric Blanket": ["Single", "Double", "Queen", "King"],
    "Electric Stove": ["Single Burner", "Double Burner"],
    "Electric Oven": ["18L", "28L", "35L"],
    "Electric Chimney": ["60cm", "90cm"],
    "Electric Geyser": ["10L", "15L", "25L"],
    "Electric Water Heater": ["3kW", "5kW"],
    "Electric Cooker": ["1L", "1.5L", "2L"],
    "Electric Deep Fryer": ["2L", "3L", "4L"],
    "Electric Sandwich Maker": ["One Size"],
    "Electric Popcorn Maker": ["One Size"],
    "Electric Coffee Maker": ["4 Cup", "6 Cup", "12 Cup"],
    "Electric Tea Maker": ["0.5L", "1L", "1.5L"],
    "Electric Milk Frother": ["One Size"],
    "Projector": ["One Size"],
    "Smart Home Hub": ["One Size"],
    "Wi-Fi Router": ["One Size"],
    "Smart Doorbell": ["One Size"],
    "Smart Lock": ["One Size"],
    "Security Camera": ["One Size"],
    # Home & Living / Furniture & Décor
    "Wall Art": ["Small", "Medium", "Large"],
    "Cushion": ["16x16 inch", "18x18 inch", "20x20 inch"],
    "Rug": ["3x5 ft", "5x8 ft", "8x10 ft"],
    "Table Lamp": ["One Size"],
    "Indoor Plant": ["Small", "Medium", "Large"],
    "Coffee Table": ["48x24 inch", "60x30 inch"],
    "Dining Table": ["4-Seater", "6-Seater"],
    "Bookshelf": ["4-Tier", "5-Tier"],
    "Office Chair": ["One Size"],
    "Sofa": ["2-Seater", "3-Seater", "Sectional"],
    "Bed Frame": ["Twin", "Full", "Queen", "King", "Cal King"],
    "Mattress": ["Twin", "Full", "Queen", "King", "Cal King"],
    "Nightstand": ["One Size"],
    "Wardrobe": ["One Size"],
    "TV Stand": ["One Size"],
    "Curtains": ["84\"", "96\"", "108\""],
    "Blinds": ["One Size"],
    "Mirror": ["One Size"],
    "Vase": ["Small", "Medium", "Large"],
    "Candles": ["Pillar", "Jar", "Taper"],
    "Throw Blanket": ["50x60 inch", "60x80 inch"],
    "Bookshelf Speaker": ["One Size"],
    "Area Heater": ["One Size"],
    "Air Purifier": ["One Size"],
    "Humidifier": ["One Size"],
    "Smart Thermostat": ["One Size"],
    "Robot Vacuum Cleaner": ["One Size"],
    "Steam Mop": ["One Size"],
    "Storage Cabinet": ["One Size"],
    # Kitchenware & Dining
    "Cutlery Set": ["16-piece", "24-piece", "32-piece"],
    "Frying Pan": ["8-inch", "10-inch", "12-inch"],
    "Dinnerware Set": ["Service for 4", "Service for 6", "Service for 8"],
    "Chef’s Knife": ["6-inch", "8-inch", "10-inch"],
    "Cutting Board": ["Small", "Medium", "Large"],
    "Mixer": ["Hand Mixer", "Stand Mixer"],
    "Stand Mixer": ["5 qt", "7 qt"],
    "Blender": ["One Size"],
    "Food Processor": ["7-cup", "12-cup"],
    "Slow Cooker": ["3 qt", "6 qt", "8 qt"],
    "Pressure Cooker": ["Stovetop", "Electric"],
    "Coffee Grinder": ["Burr", "Blade"],
    "Espresso Machine": ["Semi-Auto", "Fully Auto"],
    "Toaster": ["2-Slice", "4-Slice"],
    "Toaster Oven": ["One Size"],
    "Teapot": ["One Size"],
    "Bakeware": ["9x13 inch", "12x12 inch"],
    "Glassware": ["Set of 4", "Set of 6"],
    "Barware": ["Cocktail Shaker Set", "Wine Glass Set"],
    "Spice Rack": ["Rotating", "Wall-Mounted"],
    "Knife Block": ["Wooden", "Magnetic Strip"],
    "Dish Rack": ["One Size"],
    "Mortar & Pestle": ["Granite", "Ceramic"],
    "Rolling Pin": ["Wooden", "Silicone"],
    "Measuring Cups": ["Set of 4", "Set of 6"],
    "Measuring Spoons": ["Set of 4", "Set of 6"],
    "Dish Towels": ["Set of 2", "Set of 4"],
    "Oven Mitts": ["Pair"],
    "Ice Maker": ["One Size"],
    "Wine Cooler": ["Dual-Zone", "Single-Zone"],
    "Water Dispenser": ["Bottled", "Point-of-Use"],
    "Portable BBQ Grill": ["Charcoal", "Propane"],
    # Beauty, Personal Care & Health
    "Face Cream": ["50ml", "100ml"],
    "Lipstick": ["3g", "5g"],
    "Shampoo Bottle": ["200ml", "400ml", "600ml"],
    "Hair Serum": ["30ml", "50ml"],
    "Body Lotion": ["200ml", "500ml"],
    "Body Butter": ["200ml", "500ml"],
    "Sunscreen": ["SPF 30", "SPF 50"],
    "Perfume": ["30ml", "50ml", "100ml"],
    "Makeup Palette": ["12-color", "24-color"],
    "Foundation": ["Light", "Medium", "Full Coverage"],
    "Mascara": ["Waterproof", "Regular"],
    "Eyeliner": ["Gel", "Pencil", "Liquid"],
    "Nail Polish": ["15ml", "30ml"],
    "Electric Toothbrush": ["One Size"],
    "Flosser": ["One Size"],
    "Electric Shaver": ["One Size"],
    "Trimmer": ["One Size"],
    "Hair Dryer": ["One Size"],
    "Straightener": ["One Size"],
    "Curling Iron": ["One Size"],
    "Electric Razor": ["One Size"],
    "Epilator": ["One Size"],
    "Eyelash Curler": ["One Size"],
    "Hair Dryer Diffuser": ["One Size"],
    "Facial Cleansing Brush": ["One Size"],
    "Electric Massager": ["One Size"],
    "Foot Spa": ["One Size"],
    "First Aid Kit": ["One Size"],
    "Vitamins": ["60-count", "120-count"],
    "Supplements": ["60-count", "120-count"],
    "Fitness Tracker": ["One Size"],
    "Smart Scale": ["One Size"],
    "Yoga Mat": ["68×24 inch", "72×24 inch"],
    "Resistance Bands": ["Light", "Medium", "Heavy"],
    "Treadmill": ["One Size"],
    "Exercise Bike": ["One Size"],
    "Massage Gun": ["One Size"],
    "Yoga Blocks": ["Standard"],
    "Protein Powder": ["1 lb", "2 lb"],
    "Nutrition Bars": ["Box of 12", "Box of 24"],
    "Blood Pressure Monitor": ["One Size"],
    "Glucometer": ["One Size"],
    "Thermometer": ["Digital"],
    "Pulse Oximeter": ["One Size"],
    # Food & Beverage, Grocery & Gourmet
    "Organic Olive Oil": ["500ml", "1L"],
    "Gourmet Chocolate": ["100g", "250g"],
    "Artisanal Cheese": ["200g", "500g"],
    "Herbal Tea": ["20 Bags", "40 Bags"],
    "Coffee Beans": ["12oz", "16oz"],
    "Ground Coffee": ["12oz", "16oz"],
    "Spices": ["50g", "100g"],
    "Seasonings": ["50g", "100g"],
    "Organic Honey": ["250g", "500g"],
    "Maple Syrup": ["250ml", "500ml"],
    "Snack Packs": ["Single", "Pack of 6"],
    "Nutrition Bars": ["Box of 12", "Box of 24"],
    "Frozen Foods": ["One Size"],
    "Vegan Products": ["One Size"],
    "Plant-Based Meat": ["One Size"],
    "Health Foods": ["One Size"],
    "Supplements": ["60-count", "120-count"],
    "Craft Beer": ["6-pack", "12-pack"],
    "Wine": ["750ml", "1.5L"],
    "Spirits": ["750ml", "1L"],
    "Cold Pressed Juice": ["12oz Bottle", "16oz Bottle"],
    "Smoothie Packs": ["Pack of 5", "Pack of 10"],
    "Gourmet Snacks": ["One Size"],
    "Bakery Items": ["Loaf", "Dozen"],
    "Condiments": ["One Size"],
    "Sauces": ["One Size"],
    "Meal Kits": ["2-Person", "4-Person"],
    "Pet Food": ["2lb", "5lb"],
    # DIY & Hardware / Furniture & Lumber
    "Lawn Mower": ["One Size"],
    "Garden Hose": ["50ft", "100ft"],
    "Outdoor Furniture Set": ["4-Piece", "6-Piece"],
    "BBQ Grill": ["Small", "Medium", "Large"],
    "Electric Drill": ["One Size"],
    "Impact Driver": ["One Size"],
    "Hammer": ["16oz", "20oz"],
    "Crowbar": ["One Size"],
    "Screwdriver Set": ["6-bit", "12-bit"],
    "Bit Kit": ["20-piece", "40-piece"],
    "Measuring Tape": ["16ft", "25ft"],
    "Laser Distance Meter": ["One Size"],
    "Socket Set": ["Metric", "SAE"],
    "Wrench Set": ["Metric", "SAE"],
    "Tool Chest": ["One Size"],
    "Storage Cabinet": ["One Size"],
    "Paint": ["1 gallon", "1 quart"],
    "Paint Brushes": ["1\"", "2\"", "3\""],
    "Power Saw": ["One Size"],
    "Jigsaw": ["One Size"],
    "Nails": ["Box of 100", "Box of 500"],
    "Screws": ["Box of 100", "Box of 500"],
    "Lumber": ["2×4", "2×6", "2×8"],
    "Plywood": ["½ inch", "¾ inch"],
    "Safety Gear": ["Gloves", "Glasses", "Ear Protection"],
    "Work Light": ["One Size"],
    "Flashlight": ["One Size"],
    "Extension Cord": ["25ft", "50ft"],
    "Power Strip": ["4-Outlet", "6-Outlet"],
    "Tool Belt": ["One Size"],
    "Apron": ["One Size"],
    "Paint Rollers": ["9-inch", "18-inch"],
    "Ladder": ["6 ft", "8 ft"],
    # Office & School Supplies
    "Notebook": ["A5", "A4", "B5"],
    "Journal": ["A5", "B5"],
    "Pen Set": ["3-piece", "5-piece"],
    "Fountain Pen": ["Fine Tip", "Medium Tip"],
    "Desk Organizer": ["One Size"],
    "File Holder": ["One Size"],
    "Stapler": ["Standard", "Heavy-Duty"],
    "Staple Remover": ["One Size"],
    "Printer": ["Inkjet", "Laser"],
    "Scanner": ["Flatbed", "Sheet-Fed"],
    "Mouse Pad": ["Standard", "Extended"],
    "Wrist Rest": ["Gel", "Memory Foam"],
    "Desk Lamp": ["One Size"],
    "Task Light": ["One Size"],
    "Whiteboard": ["24×36 inch", "36×48 inch"],
    "Corkboard": ["24×36 inch", "36×48 inch"],
    "Calculator": ["Basic", "Scientific"],
    "Financial Calculator": ["One Size"],
    "File Cabinet": ["2-Drawer", "4-Drawer"],
    "Bookends": ["One Size"],
    "Binder": ["1 inch", "2 inch", "3 inch"],
    "Dividers": ["A–Z", "1–12"],
    "Backpack": ["One Size"],
    "Laptop Bag": ["13\"", "15\"", "17\""],
    "Markers": ["Permanent", "Dry-Erase"],
    "Highlighters": ["Set of 4", "Set of 6"],
    "Paper": ["A4", "A5"],
    "Envelopes": ["#10", "A2", "A7"],
    "Sticky Notes": ["3×3 inch", "4×6 inch"],
    "Index Tabs": ["One Size"],
    "Label Maker": ["One Size"],
    "Labels": ["One Size"],
    "USB Flash Drive": ["32 GB", "64 GB", "128 GB"],
    "External Hard Drive": ["1 TB", "2 TB", "4 TB"],
    "Power Bank": ["10,000mAh", "20,000mAh"],
    "Charging Station": ["4-Port", "6-Port"],
    "Webcam": ["1080p", "4K"],
    "Headset": ["Over-Ear", "In-Ear"],
    # Sports, Fitness & Outdoor
    "Yoga Mat": ["68×24 inch", "72×24 inch"],
    "Dumbbell Set": ["10 lb", "20 lb", "30 lb"],
    "Kettlebell": ["5 lb", "10 lb", "15 lb", "20 lb", "25 lb", "30 lb"],
    "Cycling Helmet": ["S", "M", "L"],
    "Protective Gear": ["Knee Pads", "Elbow Pads", "Wrist Guards"],
    "Camping Tent": ["2-Person", "4-Person", "6-Person"],
    "Sleeping Bag": ["Mummy", "Rectangular"],
    "Hiking Backpack": ["30 L", "50 L", "70 L"],
    "Trekking Poles": ["One Size"],
    "Fishing Rod": ["6 ft", "7 ft", "8 ft"],
    "Reel": ["Spinning", "Baitcasting"],
    "Golf Clubs": ["Full Set", "Half Set"],
    "Golf Bag": ["Stand Bag", "Cart Bag"],
    "Tennis Racket": ["One Size"],
    "Tennis Balls": ["Can of 3"],
    "Basketball": ["Size 7", "Size 6"],
    "Soccer Ball": ["Size 5"],
    "Camping Stove": ["One Size"],
    "Cookware": ["One Size"],
    "Inflatable Kayak": ["Single", "Tandem"],
    "Paddleboard": ["One Size"],
    "Outdoor Furniture": ["One Size"],
    "Hammock": ["One Size"],
    "Pool Accessories": ["One Size"],
    "Beach Accessories": ["One Size"],
    "Climbing Gear": ["One Size"],
    "Backpacking Gear": ["One Size"],
    "Golf Apparel": ["S", "M", "L", "XL"],
    "Golf Shoes": ["8", "9", "10", "11", "12"],
    # Toys, Games & Collectibles
    "Action Figure": ["One Size"],
    "Collectible Figurine": ["One Size"],
    "Board Game": ["Standard"],
    "Card Game": ["Standard"],
    "Puzzle": ["500-piece", "1000-piece"],
    "Jigsaw Puzzle": ["500-piece", "1000-piece"],
    "Dollhouse": ["One Size"],
    "Miniature Sets": ["One Size"],
    "Model Kit": ["One Size"],
    "RC Vehicle": ["One Size"],
    "Lego Set": ["One Size"],
    "Building Blocks": ["One Size"],
    "Stuffed Animal": ["One Size"],
    "Plush Toy": ["One Size"],
    "Educational Toy": ["One Size"],
    "STEM Kit": ["One Size"],
    "Video Game Console": ["One Size"],
    "Game Controller": ["One Size"],
    "Arcade Stick": ["One Size"],
    "VR Headset": ["One Size"],
    "Accessories": ["One Size"],
    "Trackers": ["One Size"],
    "Haptic Gloves": ["One Size"],
    "Gaming Chair": ["One Size"],
    "Racing Simulator": ["One Size"],
    "Gaming Headset": ["One Size"],
    "Microphone": ["One Size"],
    "Monitor": ["1080p", "1440p", "4K"],
    "Gaming Monitor": ["144 Hz", "240 Hz"],
    "Graphics Card": ["One Size"],
    "GPU": ["One Size"],
    "Gaming PC": ["One Size"],
    "Components": ["One Size"],
    "Gaming Laptop": ["One Size"],
    "Portable Gaming": ["One Size"],
    "Streaming Gear": ["One Size"],
    "Capture Card": ["One Size"],
    "Green Screen": ["One Size"],
    "Ring Light": ["One Size"],
    "Game Title": ["Physical", "Digital"],
    "Digital DLC": ["One Size"],
    "Game Subscription": ["One Month", "Three Months", "One Year"],
    "Online Service": ["One Month", "One Year"],
    "In-Game Currency": ["500 Credits", "1000 Credits"],
    # Automotive & Industrial
    "Car Cover": ["One Size"],
    "Sunshade": ["One Size"],
    "Steering Wheel Cover": ["One Size"],
    "Seat Covers": ["One Size"],
    "GPS Navigator": ["One Size"],
    "Dash Cam": ["One Size"],
    "Car Vacuum": ["One Size"],
    "Portable Air Compressor": ["One Size"],
    "Tire Inflator": ["One Size"],
    "Jump Starter": ["One Size"],
    "Floor Mats": ["Front Set", "Full Set"],
    "Trunk Organizer": ["One Size"],
    "Car Audio": ["One Size"],
    "Speakers": ["One Size"],
    "Motor Oil": ["1 qt", "5 qt"],
    "Fluids": ["One Size"],
    "Brake Pads": ["Front Pair", "Rear Pair"],
    "Rotors": ["One Size"],
    "Car Battery": ["Group 24", "Group 35", "Group 65"],
    "Battery Charger": ["One Size"],
    "Roof Rack": ["One Size"],
    "Cargo Carrier": ["Large", "Medium", "Small"],
    "Tow Hitch": ["Class III", "Class IV"],
    "Trailer Hitch": ["One Size"],
    "Safety Hammer": ["One Size"],
    "Seat Belt Cutter": ["One Size"],
    "Headlight Assembly": ["Left", "Right"],
    "Taillight Assembly": ["Left", "Right"],
    "Spark Plug": ["Standard", "Iridium"],
    "Ignition Coil": ["One Size"],
    "Air Filter": ["OEM", "Performance"],
    "Cabin Filter": ["Standard", "HEPA"],
    "Jumper Cables": ["4 gauge", "6 gauge"],
    "Tool Kit": ["Basic", "Advanced"],
    "Industrial Lubricants": ["Multipurpose", "High-Temp"],
    "Grease": ["One Size"],
    "3D Printer": ["One Size"],
    "Filament": ["PLA", "ABS", "TPU"],
    "CNC Accessories": ["One Size"],
    "Router Bits": ["One Size"],
    # Pet Supplies & Services
    "Dog Leash": ["S", "M", "L"],
    "Collar": ["S", "M", "L"],
    "Cat Toy": ["One Size"],
    "Laser Pointer": ["One Size"],
    "Pet Bed": ["Small", "Medium", "Large"],
    "Crate": ["Small", "Medium", "Large"],
    "Pet Food Bowl": ["One Size"],
    "Feeder": ["Automatic", "Standard"],
    "Gourmet Pet Treats": ["One Size"],
    "Grooming Kit": ["One Size"],
    "Aquarium": ["10 gal", "20 gal", "50 gal"],
    "Fish Tank": ["10 gal", "20 gal", "50 gal"],
    "Bird Cage": ["Small", "Medium", "Large"],
    "Perch": ["One Size"],
    "Small Animal Hutch": ["One Size"],
    "Cage": ["One Size"],
    "Pet Carrier": ["Small", "Medium", "Large"],
    "Travel Crate": ["Small", "Medium", "Large"],
    "Flea Preventive": ["One Size"],
    "Tick Preventive": ["One Size"],
    "Dewormer": ["One Size"],
    "Pet Clothing": ["XS", "S", "M", "L"],
    "Accessories": ["One Size"],
    "Pet ID Tag": ["One Size"],
    "Microchip Service": ["One Size"],
    "Pet Insurance": ["Monthly", "Annual"],
    "Veterinary Plan": ["Monthly", "Annual"],
    "Training Classes": ["One Size"],
    "Behavior Consultation": ["One Size"],
    # Books, Media & Digital Content
    "Paperback": ["Mass Market", "Trade"],
    "Hardcover": ["Trade", "Oversize"],
    "Audiobook": ["One Size"],
    "E-Book": ["One Size"],
    "Digital Download": ["MP3", "PDF", "ZIP"],
    "Magazine Subscription": ["Monthly", "Annual"],
    "Journal Subscription": ["Annual"],
    "Music Album": ["CD", "Vinyl", "Digital"],
    "Vinyl Record": ["33⅓ RPM", "45 RPM"],
    "Blu-ray": ["One Size"],
    "DVD": ["One Size"],
    "4K UHD": ["One Size"],
    "Streaming Service Subscription": ["Monthly", "Annual"],
    "Video Game Title": ["Physical", "Digital"],
    "Digital DLC": ["One Size"],
    "Software License": ["Single User", "Multi-User"],
    "Productivity Suite": ["Personal", "Business"],
    "Cloud Storage Subscription": ["100 GB", "1 TB", "5 TB"],
    "Online Course": ["One Size"],
    "e-Learning Platform": ["Monthly", "Annual"],
    "Virtual Conference Ticket": ["One Day", "Full Event"],
    "Webinar Pass": ["Single Webinar", "Series"],
    "Stock Photos": ["50 Images", "100 Images"],
    "Graphics Pack": ["One Size"],
    "NFT": ["One Size"],
    "Digital Collectible": ["One Size"],
    # Art Supplies & Crafting
    "Canvas Set": ["8×10", "12×12", "16×20"],
    "Easel": ["Tabletop", "Floor"],
    "Paint Brushes": ["Set of 5", "Set of 10", "Set of 20"],
    "Palette Knives": ["One Size"],
    "Acrylic Paint": ["Set of 12", "Set of 24", "Set of 48"],
    "Oil Paint": ["Set of 12", "Set of 24", "Set of 48"],
    "Watercolor Paint": ["Set of 12", "Set of 24", "Set of 48"],
    "Sketchbook": ["A4", "A3"],
    "Drawing Pad": ["A4", "A3"],
    "Watercolor Palette": ["24-Color", "48-Color"],
    "Colored Pencils": ["Set of 12", "Set of 72"],
    "Markers": ["Set of 12", "Set of 24", "Set of 48"],
    "Crafting Kit": ["One Size"],
    "DIY Project Box": ["One Size"],
    "Sewing Machine": ["One Size"],
    "Accessories": ["One Size"],
    "Scrapbooking Supplies": ["One Size"],
    "Stickers": ["One Size"],
    "Beads": ["One Size"],
    "Jewelry Making Supplies": ["One Size"],
    "Pottery Tools": ["One Size"],
    "Clay": ["Air-Dry", "Polymer", "Stoneware"],
    "3D Printer": ["One Size"],
    "Filament": ["1 kg", "500 g"],
    "Digital Drawing Tablet": ["One Size"],
    "Stylus": ["One Size"],
    "Studio Lighting": ["One Size"],
    # Baby & Toddler
    "Baby Stroller": ["One Size"],
    "Travel System": ["One Size"],
    "Diapers": ["Newborn", "Size 1", "Size 2", "Size 3", "Size 4", "Size 5", "Size 6"],
    "Pull-Ups": ["Size 2T-3T", "Size 3T-4T"],
    "Baby Monitor": ["One Size"],
    "Video Monitor": ["One Size"],
    "Baby Lotion": ["100ml", "200ml"],
    "Baby Shampoo": ["100ml", "200ml"],
    "Crib": ["One Size"],
    "Bassinet": ["One Size"],
    "Changing Table": ["One Size"],
    "Changing Pad": ["One Size"],
    "Baby Carrier": ["One Size"],
    "Wrap": ["One Size"],
    "Sling": ["One Size"],
    "High Chair": ["One Size"],
    "Booster Seat": ["One Size"],
    "Playard": ["One Size"],
    "Pack & Play": ["One Size"],
    "Toddler Toy": ["One Size"],
    "Educational Toy": ["One Size"],
    "Baby Clothes": ["0–3 Months", "3–6 Months", "6–12 Months"],
    "Bodysuits": ["0–3 Months", "3–6 Months", "6–12 Months"],
    "Pacifier": ["One Size"],
    "Teethers": ["One Size"],
    "Car Seat": ["Rear-Facing", "Convertible", "Forward-Facing"],
    "Booster Seat": ["High-Back", "Backless"],
    "Baby Bottle": ["4oz", "8oz"],
    "Sterilizer": ["One Size"],
    "Breast Pump": ["Electric Double", "Manual Single"],
    "Nursing Pillow": ["One Size"],
    "Diaper Bag": ["One Size"],
    "Backpack": ["One Size"],
    "Changing Mat": ["One Size"],
    "Baby Blanket": ["One Size"],
    "Swaddle": ["One Size"],
    "Baby Shoes": ["0–3 Months", "3–6 Months"],
    "Booties": ["0–3 Months", "3–6 Months"],
    "Safety Gate": ["One Size"],
    "Corner Guards": ["One Size"],
    # Jewelry & Watches
    "Necklace": ["One Size"],
    "Pendant": ["One Size"],
    "Earrings": ["One Size"],
    "Studs": ["One Size"],
    "Hoops": ["Small", "Medium", "Large"],
    "Bracelet": ["One Size"],
    "Bangles": ["One Size"],
    "Ring": ["US 5", "US 6", "US 7", "US 8", "US 9"],
    "Band": ["One Size"],
    "Anklet": ["One Size"],
    "Toe Ring": ["One Size"],
    "Brooch": ["One Size"],
    "Lapel Pin": ["One Size"],
    "Cufflinks": ["One Size"],
    "Tie Clip": ["One Size"],
    "Jewelry Organizer": ["One Size"],
    "Watch Winder": ["One Size"],
    "Watch Case": ["One Size"],
    # Party & Festivity
    "Party Lights": ["10 ft", "20 ft"],
    "String Lights": ["10 ft", "20 ft"],
    "Inflatable Decorations": ["One Size"],
    "Yard Inflatables": ["One Size"],
    "Birthday Banner": ["One Size"],
    "Custom Banner": ["One Size"],
    "Confetti Pack": ["Small", "Large"],
    "Tableware": ["One Size"],
    "Disposable Plates": ["7\"", "10\""],
    "Helium Tank": ["One Size"],
    "Balloons": ["Latex", "Foil"],
    "Photo Booth Props": ["One Size"],
    "Backdrops": ["One Size"],
    "Party Favors": ["One Size"],
    "Goodie Bags": ["One Size"],
    "Cake Topper": ["One Size"],
    "Dessert Stand": ["One Size"],
    "Sound System": ["One Size"],
    "DJ Equipment": ["One Size"],
    "Event Tickets": ["One Day", "Full Event"],
    "Passes": ["One Day", "Full Event"],
    # Video Games & Virtual Reality
    "Gaming Console": ["Standard Edition", "Digital Edition", "Bundle"],
    "Hardware": ["One Size"],
    "Game Controller": ["One Size"],
    "Arcade Stick": ["One Size"],
    "Racing Wheel": ["One Size"],
    "Pedals": ["One Size"],
    "VR Headset": ["One Size"],
    "Accessories": ["One Size"],
    "Trackers": ["One Size"],
    "Haptic Gloves": ["One Size"],
    "Gaming Chair": ["One Size"],
    "Racing Simulator": ["One Size"],
    "Gaming Headset": ["One Size"],
    "Microphone": ["One Size"],
    "Monitor": ["1080p", "1440p", "4K"],
    "Gaming Monitor": ["144 Hz", "240 Hz"],
    "Graphics Card": ["RTX 3060", "RTX 3070", "RTX 3080", "RX 6700 XT", "RX 6800 XT"],
    "GPU": ["One Size"],
    "Gaming PC": ["One Size"],
    "Components": ["One Size"],
    "Gaming Laptop": ["One Size"],
    "Portable Gaming": ["One Size"],
    "Streaming Gear": ["One Size"],
    "Capture Card": ["One Size"],
    "Green Screen": ["One Size"],
    "Ring Light": ["One Size"],
    "Game Title": ["Physical", "Digital"],
    "Digital DLC": ["One Size"],
    "Game Subscription": ["Monthly", "Annual"],
    "Online Service": ["Monthly", "Annual"],
    "In-Game Currency": ["500 Credits", "1000 Credits"],
    # Software, Subscriptions & Digital Services
    "Antivirus Software": ["1-Year", "2-Year"],
    "Security Software": ["1-Year", "2-Year"],
    "Photo Editing Software": ["Single License", "Family Pack"],
    "Video Editing Software": ["Single License", "Family Pack"],
    "Office Suite": ["Personal", "Business"],
    "Productivity Tools": ["One Size"],
    "Cloud Storage": ["100GB", "1TB", "5TB"],
    "Backup Service": ["One Month", "One Year"],
    "Website Hosting": ["Shared", "VPS", "Dedicated"],
    "Domain Service": [".com", ".net", ".org", "Country TLDs"],
    "VPN Subscription": ["Monthly", "Annual"],
    "Streaming Service": ["Monthly", "Annual"],
    "Digital Marketing Tool": ["Monthly", "Annual"],
    "SEO Software": ["Monthly", "Annual"],
    "Project Management Software": ["Monthly", "Annual"],
    "E-Book Subscription": ["Monthly", "Annual"],
    "Audiobook Service": ["Monthly", "Annual"],
    "VPS": ["1 GB RAM", "2 GB RAM", "4 GB RAM", "8 GB RAM"],
    "Cloud Compute": ["One Size"],
    "SaaS Tool": ["Monthly", "Annual"],
    # Eco-Friendly & Sustainable Products
    "Reusable Water Bottle": ["500ml", "750ml", "1L"],
    "Tumblers": ["One Size"],
    "Biodegradable Straws": ["Pack of 10", "Pack of 50"],
    "Cutlery": ["Pack of 4", "Pack of 8"],
    "Reusable Grocery Bags": ["One Size"],
    "Produce Bags": ["Set of 3", "Set of 5"],
    "Solar Charger": ["One Size"],
    "Solar Panel": ["One Size"],
    "Bamboo Toothbrush": ["Pack of 2", "Pack of 4"],
    "Dental Floss": ["One Size"],
    "Beeswax Wraps": ["Set of 3", "Set of 5"],
    "Reusable Food Wraps": ["Set of 3", "Set of 5"],
    "Eco-Friendly Cleaning Products": ["One Size"],
    "Compost Bin": ["One Size"],
    "Compost Tumbler": ["One Size"],
    "Organic Cotton Towels": ["Set of 2", "Set of 4"],
    "Bedsheets": ["Queen", "King", "California King"],
    "Sustainable Fashion": ["S", "M", "L", "XL"],
    "Ethical Apparel": ["S", "M", "L", "XL"],
    "Plantable Stationery": ["One Size"],
    # Luxury Goods & Premium Segments
    "Luxury Handbag": ["One Size"],
    "Designer Purse": ["One Size"],
    "High-End Watch": ["One Size"],
    "Swiss Timepiece": ["One Size"],
    "Premium Spirit": ["750ml", "1L"],
    "Aged Whiskey": ["750ml", "1L"],
    "Designer Clothing": ["S", "M", "L", "XL"],
    "Haute Couture": ["Custom Fit"],
    "Fine Jewelry": ["One Size"],
    "Gemstone Ring": ["US 5", "US 6", "US 7", "US 8", "US 9"],
    "Collector Car": ["One Size"],
    "Luxury Automobile": ["One Size"],
    "Art & Antique": ["One Size"],
    "Limited Edition Print": ["One Size"],
    "Yacht": ["One Size"],
    "Private Jet": ["One Size"],
    "Custom-Built Home": ["One Size"],
    # Recommerce & Pre-Owned / Refurbished
    "Refurbished Electronic": ["One Size"],
    "Certified Pre-Owned Device": ["One Size"],
    "Pre-Owned Luxury Bag": ["One Size"],
    "Designer Apparel": ["One Size"],
    "Used Book": ["One Size"],
    "Collectible Edition": ["One Size"],
    "Vintage Vinyl Record": ["One Size"],
    "Secondhand Instrument": ["One Size"],
    # Healthcare & Medical Supplies
    "Medical Mask": ["N95", "KN95"],
    "Respirator": ["N95", "KN95"],
    "Hand Sanitizer": ["250ml", "500ml"],
    "Disinfectant Wipes": ["Pack of 50", "Pack of 100"],
    "First Aid Kit": ["One Size"],
    "Blood Pressure Monitor": ["One Size"],
    "Glucometer": ["One Size"],
    "Wheelchair": ["One Size"],
    "Mobility Aid": ["One Size"],
    "CPAP Machine": ["One Size"],
    "Sleep Apnea Device": ["One Size"],
    "Diagnostic Test Kit": ["One Size"],
    "Orthopedic Support": ["One Size"],
    "Braces": ["One Size"],
    "Prescription Medication": ["One Month", "Three Months"],
    "Telemedicine Service": ["One Consultation", "Monthly Package"],
    # Subscriptions & Memberships
    "Meal Kit Delivery": ["2-Person", "4-Person"],
    "Prepared Meal Plan": ["Weekly", "Monthly"],
    "Streaming Video": ["Monthly", "Annual"],
    "Streaming Music": ["Monthly", "Annual"],
    "Subscription Box": ["Monthly", "Quarterly"],
    "Gym Membership": ["Monthly", "Annual"],
    "Digital Publication": ["Monthly", "Annual"],
    "Professional Network": ["Monthly", "Annual"],
    "Gaming Subscription": ["Monthly", "Annual"],
    "Online Service": ["Monthly", "Annual"],    
    }

    # Get the Product details (name, description & details)
    product_details = get_product_detail(product_name, product_descriptions, product_sizes, color)
    product_description = product_details["Product_Description"]
    product_size = product_details["Product_Size"]
    product_color = product_details["Product_color"]
    
    
    # PRODUCTS
    avg_inventory = random.randint(1, 100)
    #c_avg_inventory = product_inventory_adjust(avg_inventory)
    
    days_since_creation = random.randint(1, 360)
    #c_days_since_creation = product_age_adjust(days_since_creation)
    
    images_count = random.randint(0, 6)
    #c_images_count = produt_image_adjust(images_count)
    
    product_type = random.choice(["seasonal", "regular"])
    #c_product_type = product_type_adjust(product_type)
    
    product_tag = random.choice(["winter", "summer", "spring", "autumn", "electronics", "luxury", "casual"])
    #c_product_tag = product_tag_adjust(product_tag)
    
    product_price = random.randint(199, 9000)
    #c_product_price = avg_price_adjust(product_price)
    
    # Dashboard Data Clint Minimum Price
    min_price_client, min_price_dicount = get_min_price_from_product_price(product_price)


    # USER
    orders = random.randint(0, 12)
    #c_orders = user_order_score(orders)
    
    spent = random.randint(50, 10000)
    #c_spent = user_spending_adjust(spent)
    
    refunded = random.randint(0, 600)
   # c_refunded = user_refund_penalty(refunded, spent)
    
    days_since_last = random.randint(1, 400)
    #c_days_since_last = user_recency_bonus(days_since_last)

    #c_verified = user_email_verify_adjust(user_email)

    # User, Product, Bargain Scores.
    user_score = calculate_user_score(orders, refunded, spent, days_since_last, user_email)
    product_score = calculate_product_score(days_since_creation, avg_inventory, images_count, product_type, product_tag, product_price)
    bargain_score = random.randint(30, 100)

# measure ments taken in current_round_count iteration     
    # Round Counts (user, product, bargain)
    max_round_count = calculate_max_rounds(user_score, product_score, bargain_score)
    current_round = random.randint(1, max_round_count)
    
    adjusted_discount = adjust_max_discount(min_price_dicount, user_score, product_score, bargain_score)
    round_counts_with_discounts = generate_all_round_discounts(max_round_count, adjusted_discount)
    
## current round, over all round and discount logical pipeline 
    previous_discount_given, user_req = get_previous_round(current_round, round_counts_with_discounts)
    
    # -----------------------------------------------------------------------------------

    # Get discount for current round
    current_discount = get_discount_for_current_round(current_round, round_counts_with_discounts)
    
    discounted_price_current_round = round(get_original_price_from_discounts(product_price, current_discount), 2)
    
    if current_round >= 3 :
        response_message = (
           f"Bot: You've bargained well, so here's my final offer {current_discount}% off, final price {discounted_price_current_round} INR. Deal?"
        )
    else:
        response_message = (
             f"Bot: I’d love to work something out how about {current_discount}% off, making it {discounted_price_current_round} INR? I'm open to hearing your offer too!"
        )

    Data_from_data_base = (
        f"User Info:\n"
        f"user_name = {user_name}, email = {user_email}, "
        f"orders = {orders}, spent = {spent}, email_verified = {user_email}, refunds = {refunded}, days_since_last = {days_since_last}\n\n"

        f"Product Info:\n"
        f"product_name = {product_name}, product_description = {product_description}, product_color = {product_color}, product_size = {product_size}, "
        f"product_price = {product_price} INR, min_price_client = {min_price_client} INR, avg_inventory = {avg_inventory}, "
        f"product_tag = {product_tag}, product_type = {product_type}, images_count = {images_count}, days_since_creation = {days_since_creation}\n\n"   
    )

    user = (
#        f"User: along with that text {previous_roudn_discount} and {user_req_discount} prep this U_discoun from the previous discount ?"
        f"User: {previous_discount_given}% is less it seems is that all you can offer,ok lets say if you can make it to {user_req} thats a deal"
    )

    response = response_message


    n_user = normalize_text(user)
    n_response = normalize_text(response)
    n_Data_from_data_base = normalize_text(Data_from_data_base)

    return {
       "Live_data_from_frontend": n_Data_from_data_base,
        "user": n_user,
        "response": n_response,
        "metadata": {
            # User
            "user_email_id": user_email,
            "user_score": user_score,
            # Bargain 
            "bargain_score": bargain_score,
            # Product
            "product_price": product_price,
            "product_score": product_score,
            "admin_max_discount_from_min_price": min_price_dicount,
            "system_max_discount": adjusted_discount,
            "max_round_count": max_round_count,
            #"lower_bound": lower_bound_cr,
            "current_round": current_round,
            "previous_round_discount_given" : previous_discount_given,
            "user_req": user_req,
            "current_discount": current_discount,
            "current_discount_price": discounted_price_current_round,
            "round_discounts": round_counts_with_discounts
        }
    }

In [ ]:
# file save

import json
from tqdm import trange  # Install with `pip install tqdm` if needed

# Dataset Generator with progress bar and error handling
def generate_dataset(n=100, filename="/mnt/data/bargenix_single_turn_full_500.jsonl"):
    successful = 0
    with open(filename, "w") as f:
        for i in trange(n, desc="Generating Samples"):
            try:
                sample = generate_scenario()
                f.write(json.dumps(sample) + "\n")
                successful += 1
            except Exception as e:
                print(f"⚠️ Error generating sample {i + 1}: {e}")
    print(f"✅ Successfully generated {successful}/{n} samples → {filename}")


In [15]:
# generate_dataset() for Jupyter (no file saving)

import json
from tqdm import trange  # For optional progress bar display

def n_generate_datasets(n=50):
    dataset = []
    for i in trange(n, desc="Generating Samples"):
        try:
            sample = generate_scenario()
            dataset.append(sample)
        except Exception as e:
            print(f"⚠️ Error generating sample {i + 1}: {e}")
    
    # Print as formatted JSON
    print(json.dumps(dataset, indent=2))
    
    return dataset  # Optional: Return the dataset if you want to work with it later


In [16]:
data = n_generate_datasets(n=5)

Generating Samples: 100%|██████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 2059.46it/s]

[
  {
    "Live_data_from_frontend": "User Info:\nuser_name = Girish, email = girish@gmail.com, orders = 0, spent = 4684, email_verified = girish@gmail.com, refunds = 215, days_since_last = 85\n\nProduct Info:\nproduct_name = Espresso Machine, product_description = Semi-automatic or fully automatic espresso machine with PID temperature control., product_color = Mauve, product_size = Fully Auto, product_price = 6432 INR, min_price_client = 3538 INR, avg_inventory = 3, product_tag = winter, product_type = regular, images_count = 2, days_since_creation = 87\n\n",
    "user": "User: 18% is less it seems is that all you can offer,ok lets say if you can make it to 39 thats a deal",
    "response": "Bot: You've bargained well, so here's my final offer20% off, final price 5145.6 INR. Deal?",
    "metadata": {
      "user_email_id": "girish@gmail.com",
      "user_score": 23.808274133449494,
      "bargain_score": 71,
      "product_price": 6432,
      "product_score": 31.588528194400546,
     